In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np
import dgl
from dgl.nn import GATConv

/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/dgl/backend/pytorch/sparse.py:104: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=th.float16)
/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/dgl/backend/pytorch/sparse.py:128: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dZ):
/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/dgl/backend/pytorch/sparse.py:177: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=th.float16)
/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/dgl/backend/pytorch/sparse.py:207: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is

In [12]:


df = pd.read_csv("raw_kg.tsv.gz", sep="\t", compression="gzip")

print(df.head())

      x_id x_type    x_name  x_source            y_id   y_type  \
0    93611   Gene    FBXO44      NCBI  UBERON:0000082  Anatomy   
1    53343   Gene     NUDT9      NCBI  UBERON:0001302  Anatomy   
2   219793   Gene     TBATA      NCBI  UBERON:0001225  Anatomy   
3  DB00682   Drug  Warfarin  DrugBank         DB09073     Drug   
4     9732   Gene     DOCK4      NCBI  UBERON:0002167  Anatomy   

                   y_name  y_source       relation         display_relation  \
0  adult mammalian kidney    UBERON      ExpressIN   Gene_ExpressIN_Anatomy   
1      right uterine tube    UBERON      ExpressIN   Gene_ExpressIN_Anatomy   
2        cortex of kidney    UBERON  LowExpression  Gene_LowExpress_Anatomy   
3             Palbociclib  DrugBank            DDI    Drug_Drug_Interaction   
4              right lung    UBERON      ExpressIN   Gene_ExpressIN_Anatomy   

  rel_source  
0       Bgee  
1       Bgee  
2       Bgee  
3   DrugBank  
4       Bgee  


In [13]:
filtered_df = df[df['display_relation'].isin(['Gene_SL_Gene', 'Gene_NONSL_Gene'])]
filtered_df

,x_id,x_type,x_name,x_source,y_id,y_type,y_name,y_source,relation,display_relation,rel_source
21,5440,Gene,POLR2K,NCBI,30834,Gene,POLR1H,NCBI,NONSL,Gene_NONSL_Gene,High Throughput
23,1576,Gene,CYP3A4,NCBI,84310,Gene,C7orf50,NCBI,SL,Gene_SL_Gene,GCATSL;KG4SL
31,25862,Gene,USP49,NCBI,81876,Gene,RAB1B,NCBI,SL,Gene_SL_Gene,NSF4SL
37,80820,Gene,EEPD1,NCBI,284532,Gene,OR14A16,NCBI,SL,Gene_SL_Gene,SL2MF
38,207,Gene,AKT1,NCBI,6223,Gene,RPS19,NCBI,SL,Gene_SL_Gene,DDGCN;NSF4SL
...,...,...,...,...,...,...,...,...,...,...,...
12278422,1441,Gene,CSF3R,NCBI,23560,Gene,GTPBP4,NCBI,SL,Gene_SL_Gene,GRSMF;PTGNN;DDGCN
12278431,5428,Gene,POLG,NCBI,2967,Gene,GTF2H3,NCBI,SL,Gene_SL_Gene,SLMGAE
12278432,3014,Gene,H2AX,NCBI,6647,Gene,SOD1,NCBI,SL,Gene_SL_Gene,SL2MF;SLGNN;NSF4SL
12278433,5742,Gene,PTGS1,NCBI,89801,Gene,PPP1R3F,NCBI,SL,Gene_SL_Gene,KG4SL;SLGNN;GCATSL;PTGNN


In [14]:
triples_df = filtered_df[['x_name', 'display_relation', 'y_name']]
triples_df.columns = ['head', 'relation', 'tail']
triples_path = "triples.tsv"
triples_df.to_csv(triples_path, sep="\t", index=False, header=False)

In [15]:
# 1. Load and preprocess the dataset
triples_path = "triples.tsv"  # Path to your triples file
triples_df = pd.read_csv(triples_path, sep="\t", names=["head", "relation", "tail"])

# Check the relation counts to confirm the names
print(triples_df['relation'].value_counts())

# Filter the dataset to include only the relations 'Gene_SL_Gene' and 'Gene_NONSL_Gene'
filtered_df = triples_df[triples_df['relation'].isin(['Gene_SL_Gene', 'Gene_NONSL_Gene'])]

# Check if the filtered dataframe has data
print(f"Filtered DataFrame Size: {len(filtered_df)} rows")

relation
Gene_SL_Gene       1809140
Gene_NONSL_Gene     112246
Name: count, dtype: int64
Filtered DataFrame Size: 1921386 rows


In [16]:
# Create entity-to-index mapping
unique_entities = set(filtered_df['head']).union(set(filtered_df['tail']))
entity_to_idx = {entity: idx for idx, entity in enumerate(unique_entities)}

# Create relation-to-index mapping
unique_relations = set(filtered_df['relation'])
relation_to_idx = {relation: idx for idx, relation in enumerate(unique_relations)}

# Create the graph
def create_graph_from_triples(triples_df, entity_to_idx, relation_to_idx):
    head_idx = [entity_to_idx[entity] for entity in triples_df['head']]
    tail_idx = [entity_to_idx[entity] for entity in triples_df['tail']]
    graph = dgl.graph((head_idx, tail_idx))
    graph.edata['relation'] = torch.tensor([relation_to_idx[relation] for relation in triples_df['relation']])
    
    # Add self-loops to resolve zero in-degree issue
    graph = dgl.add_self_loop(graph)
    
    return graph

# Create the graph
graph = create_graph_from_triples(filtered_df, entity_to_idx, relation_to_idx)


In [17]:
# 4. Dataset class
class SLDataset(Dataset):
    def __init__(self, triples_df, entity_to_idx):
        self.triples = triples_df
        self.entity_to_idx = entity_to_idx

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        row = self.triples.iloc[idx]
        head_idx = self.entity_to_idx[row['head']]
        tail_idx = self.entity_to_idx[row['tail']]
        return torch.tensor(head_idx), torch.tensor(tail_idx), torch.tensor(1.0)

In [18]:
train_df, test_df = train_test_split(filtered_df, test_size=0.2)

train_dataset = SLDataset(train_df, entity_to_idx)
test_dataset = SLDataset(test_df, entity_to_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Load and preprocess the dataset
triples_path = "triples.tsv"  # Path to your triples file
triples_df = pd.read_csv(triples_path, sep="\t", names=["head", "relation", "tail"])

# Check the relation counts to confirm the names
print(triples_df['relation'].value_counts())

# Filter the dataset to include only the relations 'Gene_SL_Gene' and 'Gene_NONSL_Gene'
filtered_df = triples_df[triples_df['relation'].isin(['Gene_SL_Gene', 'Gene_NONSL_Gene'])]

# Check if the filtered dataframe has data
print(f"Filtered DataFrame Size: {len(filtered_df)} rows")

# 2. Create the DGL Graph

# Create entity-to-index mapping
unique_entities = set(filtered_df['head']).union(set(filtered_df['tail']))
entity_to_idx = {entity: idx for idx, entity in enumerate(unique_entities)}

# Create relation-to-index mapping
unique_relations = set(filtered_df['relation'])
relation_to_idx = {relation: idx for idx, relation in enumerate(unique_relations)}

# Create the graph
def create_graph_from_triples(triples_df, entity_to_idx, relation_to_idx):
    head_idx = [entity_to_idx[entity] for entity in triples_df['head']]
    tail_idx = [entity_to_idx[entity] for entity in triples_df['tail']]
    graph = dgl.graph((head_idx, tail_idx))
    graph.edata['relation'] = torch.tensor([relation_to_idx[relation] for relation in triples_df['relation']])
    
    # Add self-loops to resolve zero in-degree issue
    graph = dgl.add_self_loop(graph)
    
    return graph

# Create the graph
graph = create_graph_from_triples(filtered_df, entity_to_idx, relation_to_idx)

# Move graph to the same device as model
graph = graph.to(device)

class GATLayer(nn.Module):
    def __init__(self, in_dim, out_dim, num_heads, dropout):
        super(GATLayer, self).__init__()
        self.attn = GATConv(in_dim, out_dim, num_heads, feat_drop=dropout, attn_drop=dropout, negative_slope=0.2)
        self.fc = nn.Linear(in_dim, out_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, graph, node_features):
        # Only one output now from GATConv, so no need for unpacking
        out = self.attn(graph, node_features)  # This will be the output tensor
        out = self.fc(out)
        return self.dropout(out)


relation
Gene_SL_Gene       1809140
Gene_NONSL_Gene     112246
Name: count, dtype: int64
Filtered DataFrame Size: 1921386 rows


In [21]:
class SLGNNModel(nn.Module):
    def __init__(self, n_entities, n_relations, n_factors, emb_dim=64):
        super(SLGNNModel, self).__init__()

        # Embedding layers for entities and relations
        self.entity_embeddings = nn.Embedding(n_entities, emb_dim)
        self.relation_embeddings = nn.Embedding(n_relations, emb_dim)

        # Latent factors (if applicable)
        self.latent_factors = nn.Parameter(torch.randn(n_factors, emb_dim))

        # Graph convolution layers
        self.gat_layer = GATLayer(emb_dim, emb_dim, num_heads=2, dropout=0.2)

        # Output layer for binary classification (logits)
        self.fc = nn.Linear(emb_dim, 1)

    def forward(self, graph, gene_a, gene_b):
        # Get entity embeddings
        entity_emb = self.entity_embeddings.weight  # Get all entity embeddings

        # Forward pass through GAT layers
        gene_embeddings = self.gat_layer(graph, entity_emb)

        # Flatten the embeddings to 2D for pairwise similarity calculation
        gene_embeddings = gene_embeddings.view(gene_embeddings.size(0), -1)  # Flatten

        # Compute pairwise similarity (dot product) between gene_a and gene_b
        logits = torch.sum(gene_embeddings[gene_a] * gene_embeddings[gene_b], dim=1, keepdim=True)  # Single output

        return logits




In [22]:
# 6. Model Initialization
n_entities = len(entity_to_idx)
n_relations = len(set(filtered_df['relation']))
n_factors = 4
emb_dim = 64
num_epochs = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SLGNNModel(n_entities, n_relations, n_factors, emb_dim).to(device)

In [ ]:
# 7. Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [16]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix


n_entities = len(entity_to_idx)
n_relations = len(set(filtered_df['relation']))
n_factors = 4
emb_dim = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SLGNNModel(n_entities, n_relations, n_factors, emb_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 6. Class balancing with pos_weight for BCE loss
pos_weight = torch.tensor([len(filtered_df) / filtered_df['relation'].value_counts()['Gene_SL_Gene']]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# 7. Training loop with confusion matrix and relations tracking
for epoch in range(1, 2):  # Run for 1 epoch for now, can increase
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []

    relations_built = set()  # To track unique relations built in each epoch

    for gene_a, gene_b, labels in train_loader:
        gene_a = gene_a.to(device)
        gene_b = gene_b.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass with the graph
        logits = model(graph, gene_a, gene_b)

        # Compute loss
        loss = criterion(logits.squeeze(), labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Track relations
        for idx in range(len(gene_a)):
            edge_id = graph.edge_ids(gene_a[idx], gene_b[idx])
            relation = graph.edata['relation'][edge_id].item()
            relations_built.add(relation)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(torch.sigmoid(logits).detach().cpu().numpy())  # Detach before numpy()

    # Convert all_preds to binary (0 or 1) for confusion matrix
    all_preds = np.array(all_preds) > 0.5  # Apply threshold of 0.5

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Print or log the results
    print(f"Epoch {epoch}:")
    print(f"Loss: {total_loss / len(train_loader)}")
    print(f"Relations built this epoch: {len(relations_built)}")
    print(f"Confusion Matrix:\n{cm}")

Epoch 1:
Loss: 0.0018354626520599444
Relations built this epoch: 2
Confusion Matrix:
[[      0       0]
 [     84 1537024]]


In [18]:
 #5. Class balancing with compute_class_weight
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights based on the relation labels
class_weights = compute_class_weight('balanced', classes=np.unique(filtered_df['relation']), y=filtered_df['relation'])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])  # Gene_NONSL_Gene is the minority class

# 6. Training loop with confusion matrix and relations tracking
n_entities = len(entity_to_idx)
n_relations = len(set(filtered_df['relation']))
n_factors = 4
emb_dim = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SLGNNModel(n_entities, n_relations, n_factors, emb_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 2):  # Run for 1 epoch for now, can increase
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []

    relations_built = set()  # To track unique relations built in each epoch

    for gene_a, gene_b, labels in train_loader:
        gene_a = gene_a.to(device)
        gene_b = gene_b.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        logits = model(graph, gene_a, gene_b)

        loss = criterion(logits.squeeze(), labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Track relations
        for idx in range(len(gene_a)):
            edge_id = graph.edge_ids(gene_a[idx], gene_b[idx])  # Updated edge_ids method
            relation = graph.edata['relation'][edge_id].item()
            relations_built.add(relation)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(torch.sigmoid(logits).detach().cpu().numpy())  # Detach before numpy()

    all_preds = np.array(all_preds) > 0.5  # Convert predictions to binary

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Print or log the results
    print(f"Epoch {epoch}:")
    print(f"Loss: {total_loss / len(train_loader)}")
    print(f"Relations built this epoch: {len(relations_built)}")
    print(f"Confusion Matrix:\n{cm}")

Epoch 1:
Loss: 11.100265499381603
Relations built this epoch: 2
Confusion Matrix:
[[      0       0]
 [    125 1536983]]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
import dgl
from dgl.nn import GATConv
from torch.utils.data import Dataset, DataLoader

# 1. Load and preprocess the dataset
triples_path = "triples.tsv"  # Path to your triples file
triples_df = pd.read_csv(triples_path, sep="\t", names=["head", "relation", "tail"])

# Filter the dataset to include only the relations 'Gene_SL_Gene' and 'Gene_NONSL_Gene'
filtered_df = triples_df[triples_df['relation'].isin(['Gene_SL_Gene', 'Gene_NONSL_Gene'])]

# Create entity-to-index mapping
unique_entities = set(filtered_df['head']).union(set(filtered_df['tail']))
entity_to_idx = {entity: idx for idx, entity in enumerate(unique_entities)}

# Create relation-to-index mapping
unique_relations = set(filtered_df['relation'])
relation_to_idx = {relation: idx for idx, relation in enumerate(unique_relations)}

# Create the graph
def create_graph_from_triples(triples_df, entity_to_idx, relation_to_idx):
    head_idx = [entity_to_idx[entity] for entity in triples_df['head']]
    tail_idx = [entity_to_idx[entity] for entity in triples_df['tail']]
    graph = dgl.graph((head_idx, tail_idx))
    graph.edata['relation'] = torch.tensor([relation_to_idx[relation] for relation in triples_df['relation']])
    
    # Add self-loops to resolve zero in-degree issue
    graph = dgl.add_self_loop(graph)
    
    return graph

# Create the graph
graph = create_graph_from_triples(filtered_df, entity_to_idx, relation_to_idx)

# Move graph to the same device as model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
graph = graph.to(device)

# 2. Dataset class
class SLDataset(Dataset):
    def __init__(self, triples_df, entity_to_idx):
        self.triples = triples_df
        self.entity_to_idx = entity_to_idx

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        row = self.triples.iloc[idx]
        head_idx = self.entity_to_idx[row['head']]
        tail_idx = self.entity_to_idx[row['tail']]
        return torch.tensor(head_idx), torch.tensor(tail_idx), torch.tensor(1.0)  # Assuming binary classification

train_df, test_df = train_test_split(filtered_df, test_size=0.2)

train_dataset = SLDataset(train_df, entity_to_idx)
test_dataset = SLDataset(test_df, entity_to_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. GAT Layer and SLGNN Model
class GATLayer(nn.Module):
    def __init__(self, in_dim, out_dim, num_heads, dropout):
        super(GATLayer, self).__init__()
        self.attn = GATConv(in_dim, out_dim, num_heads, feat_drop=dropout, attn_drop=dropout, negative_slope=0.2)
        self.fc = nn.Linear(in_dim, out_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, graph, node_features):
        out = self.attn(graph, node_features)
        out = self.fc(out)
        return self.dropout(out)

class SLGNNModel(nn.Module):
    def __init__(self, n_entities, n_relations, n_factors, emb_dim=64):
        super(SLGNNModel, self).__init__()

        # Embedding layers for entities and relations
        self.entity_embeddings = nn.Embedding(n_entities, emb_dim)
        self.relation_embeddings = nn.Embedding(n_relations, emb_dim)

        # Latent factors (if applicable)
        self.latent_factors = nn.Parameter(torch.randn(n_factors, emb_dim))

        # Graph convolution layers
        self.gat_layer = GATLayer(emb_dim, emb_dim, num_heads=2, dropout=0.2)

        self.fc = nn.Linear(emb_dim, 1)  # Output layer for binary classification

    def forward(self, graph, gene_a, gene_b):
        entity_emb = self.entity_embeddings.weight  # Get all entity embeddings
        gene_embeddings = self.gat_layer(graph, entity_emb)
        gene_embeddings = gene_embeddings.view(gene_embeddings.size(0), -1)  # Flatten

        # Compute pairwise similarity (dot product) between gene_a and gene_b
        logits = torch.sum(gene_embeddings[gene_a] * gene_embeddings[gene_b], dim=1, keepdim=True)
        return logits

# 4. Class balancing with compute_class_weight
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights based on the relation labels
class_weights = compute_class_weight('balanced', classes=np.unique(filtered_df['relation']), y=filtered_df['relation'])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])  # Gene_NONSL_Gene is the minority class

# 5. Training loop with confusion matrix and relations tracking
n_entities = len(entity_to_idx)
n_relations = len(set(filtered_df['relation']))
n_factors = 4
emb_dim = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SLGNNModel(n_entities, n_relations, n_factors, emb_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Lower learning rate for better convergence

for epoch in range(1, 11):  # Run for 10 epochs instead of just 1
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []

    relations_built = set()  # To track unique relations built in each epoch

    for gene_a, gene_b, labels in train_loader:
        gene_a = gene_a.to(device)
        gene_b = gene_b.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        logits = model(graph, gene_a, gene_b)

        loss = criterion(logits.squeeze(), labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Track relations
        for idx in range(len(gene_a)):
            edge_id = graph.edge_ids(gene_a[idx], gene_b[idx])  # Updated edge_ids method
            relation = graph.edata['relation'][edge_id].item()
            relations_built.add(relation)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(torch.sigmoid(logits).detach().cpu().numpy())  # Detach before numpy()

    all_preds = np.array(all_preds) > 0.5  # Convert predictions to binary

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Compute Precision, Recall, F1 Score, and ROC-AUC
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    roc_auc = roc_auc_score(all_labels, all_preds)

    # Print or log the results
    print(f"Epoch {epoch}:")
    print(f"Loss: {total_loss / len(train_loader)}")
    print(f"Relations built this epoch: {len(relations_built)}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC-AUC: {roc_auc}")


/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \
/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Epoch 1:
Loss: 0.00027135195155561737
Relations built this epoch: 2
Confusion Matrix:
[[      0       0]
 [    169 1536939]]
Precision: 1.0
Recall: 0.9998900532688659
F1 Score: 0.999945023612196
ROC-AUC: nan


/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Epoch 2:
Loss: 0.00039450027945920154
Relations built this epoch: 2
Confusion Matrix:
[[      0       0]
 [     30 1537078]]
Precision: 1.0
Recall: 0.9999804828287928
F1 Score: 0.9999902413191655
ROC-AUC: nan


/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


...

Epoch 5:
Loss: 0.06302720048716205
Relations built this epoch: 2
Confusion Matrix:
[[      0       0]
 [     65 1537043]]
Precision: 1.0
Recall: 0.9999577127957177
F1 Score: 0.9999788559507975
ROC-AUC: nan


/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Epoch 6:
Loss: 0.22412489104541058
Relations built this epoch: 2
Confusion Matrix:
[[      0       0]
 [     69 1537039]]
Precision: 1.0
Recall: 0.9999551105062233
F1 Score: 0.9999775547493337
ROC-AUC: nan


/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Epoch 7:
Loss: 1.0739943599823774
Relations built this epoch: 2
Confusion Matrix:
[[      0       0]
 [     90 1537018]]
Precision: 1.0
Recall: 0.9999414484863783
F1 Score: 0.9999707233860942
ROC-AUC: nan


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
import dgl
from dgl.nn import GATConv
from torch.utils.data import Dataset, DataLoader
from imblearn.over_sampling import RandomOverSampler

# 1. Load and preprocess the dataset
triples_path = "triples.tsv"  # Path to your triples file
triples_df = pd.read_csv(triples_path, sep="\t", names=["head", "relation", "tail"])

# Filter the dataset to include only the relations 'Gene_SL_Gene' and 'Gene_NONSL_Gene'
filtered_df = triples_df[triples_df['relation'].isin(['Gene_SL_Gene', 'Gene_NONSL_Gene'])]

# Oversample the minority class (Gene_NONSL_Gene)
ros = RandomOverSampler(sampling_strategy='minority')
X_resampled, y_resampled = ros.fit_resample(filtered_df[['head', 'tail']], filtered_df['relation'])

# Create a new DataFrame after oversampling
resampled_df = pd.DataFrame(X_resampled, columns=['head', 'tail'])
resampled_df['relation'] = y_resampled

# Create entity-to-index mapping
unique_entities = set(resampled_df['head']).union(set(resampled_df['tail']))
entity_to_idx = {entity: idx for idx, entity in enumerate(unique_entities)}

# Create relation-to-index mapping
unique_relations = set(resampled_df['relation'])
relation_to_idx = {relation: idx for idx, relation in enumerate(unique_relations)}

# Create the graph
def create_graph_from_triples(triples_df, entity_to_idx, relation_to_idx):
    head_idx = [entity_to_idx[entity] for entity in triples_df['head']]
    tail_idx = [entity_to_idx[entity] for entity in triples_df['tail']]
    graph = dgl.graph((head_idx, tail_idx))
    graph.edata['relation'] = torch.tensor([relation_to_idx[relation] for relation in triples_df['relation']])
    
    # Add self-loops to resolve zero in-degree issue
    graph = dgl.add_self_loop(graph)
    
    return graph

# Create the graph
graph = create_graph_from_triples(resampled_df, entity_to_idx, relation_to_idx)

# Move graph to the same device as model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
graph = graph.to(device)

# Dataset class
class SLDataset(Dataset):
    def __init__(self, triples_df, entity_to_idx):
        self.triples = triples_df
        self.entity_to_idx = entity_to_idx

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        row = self.triples.iloc[idx]
        head_idx = self.entity_to_idx[row['head']]
        tail_idx = self.entity_to_idx[row['tail']]
        return torch.tensor(head_idx), torch.tensor(tail_idx), torch.tensor(1.0)  # Assuming binary classification

train_df, test_df = train_test_split(resampled_df, test_size=0.2)

train_dataset = SLDataset(train_df, entity_to_idx)
test_dataset = SLDataset(test_df, entity_to_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# GAT Layer and SLGNN Model
class GATLayer(nn.Module):
    def __init__(self, in_dim, out_dim, num_heads, dropout):
        super(GATLayer, self).__init__()
        self.attn = GATConv(in_dim, out_dim, num_heads, feat_drop=dropout, attn_drop=dropout, negative_slope=0.2)
        self.fc = nn.Linear(in_dim, out_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, graph, node_features):
        out = self.attn(graph, node_features)
        out = self.fc(out)
        return self.dropout(out)

class SLGNNModel(nn.Module):
    def __init__(self, n_entities, n_relations, n_factors, emb_dim=64):
        super(SLGNNModel, self).__init__()

        # Embedding layers for entities and relations
        self.entity_embeddings = nn.Embedding(n_entities, emb_dim)
        self.relation_embeddings = nn.Embedding(n_relations, emb_dim)

        # Latent factors (if applicable)
        self.latent_factors = nn.Parameter(torch.randn(n_factors, emb_dim))

        # Graph convolution layers
        self.gat_layer = GATLayer(emb_dim, emb_dim, num_heads=2, dropout=0.2)

        self.fc = nn.Linear(emb_dim, 1)  # Output layer for binary classification

    def forward(self, graph, gene_a, gene_b):
        entity_emb = self.entity_embeddings.weight  # Get all entity embeddings
        gene_embeddings = self.gat_layer(graph, entity_emb)
        gene_embeddings = gene_embeddings.view(gene_embeddings.size(0), -1)  # Flatten

        # Compute pairwise similarity (dot product) between gene_a and gene_b
        logits = torch.sum(gene_embeddings[gene_a] * gene_embeddings[gene_b], dim=1, keepdim=True)
        return logits

# Class balancing using computed weights
class_weights = compute_class_weight('balanced', classes=np.unique(resampled_df['relation']), y=resampled_df['relation'])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])  # Gene_NONSL_Gene is the minority class

# Training loop
n_entities = len(entity_to_idx)
n_relations = len(set(resampled_df['relation']))
n_factors = 4
emb_dim = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SLGNNModel(n_entities, n_relations, n_factors, emb_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Lower learning rate for better convergence

for epoch in range(1, 2):  # Run for 10 epochs instead of just 1
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []

    relations_built = set()

    for gene_a, gene_b, labels in train_loader:
        gene_a = gene_a.to(device)
        gene_b = gene_b.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        logits = model(graph, gene_a, gene_b)

        loss = criterion(logits.squeeze(), labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Track relations
        for idx in range(len(gene_a)):
            edge_id = graph.edge_ids(gene_a[idx], gene_b[idx])  # Updated edge_ids method
            relation = graph.edata['relation'][edge_id].item()
            relations_built.add(relation)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(torch.sigmoid(logits).detach().cpu().numpy())  # Detach before numpy()

    all_preds = np.array(all_preds) > 0.5  # Convert predictions to binary

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Compute Precision, Recall, F1 Score, and ROC-AUC
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    roc_auc = roc_auc_score(all_labels, all_preds)

    # Print or log the results
    print(f"Epoch {epoch}:")
    print(f"Loss: {total_loss / len(train_loader)}")
    print(f"Relations built this epoch: {len(relations_built)}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC-AUC: {roc_auc}")


/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \


Epoch 1:
Loss: 0.00040941086307705684
Relations built this epoch: 2
Confusion Matrix:
[[      0       0]
 [    286 2894338]]
Precision: 1.0
Recall: 0.9999011961484462
F1 Score: 0.9999505956335523
ROC-AUC: nan


/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [23]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
import numpy as np

# Assuming filtered_df is your original DataFrame with 'head', 'tail', and 'relation'

# Separate the features (head, tail) and labels (relation)
X = filtered_df[['head', 'tail']]
y = filtered_df['relation']

# Initialize RandomOverSampler
ros = RandomOverSampler(sampling_strategy='minority')  # Only oversample the minority class

# Apply oversampling
X_resampled, y_resampled = ros.fit_resample(X, y)

# Create a new DataFrame with the resampled data
resampled_df = pd.DataFrame(X_resampled, columns=['head', 'tail'])
resampled_df['relation'] = y_resampled

# Check the distribution of the classes in the resampled dataset
print(resampled_df['relation'].value_counts())

# Now, use resampled_df instead of filtered_df for further steps


relation
Gene_NONSL_Gene    1809140
Gene_SL_Gene       1809140
Name: count, dtype: int64


In [26]:
# Create entity-to-index mapping
unique_entities = set(resampled_df['head']).union(set(resampled_df['tail']))
entity_to_idx = {entity: idx for idx, entity in enumerate(unique_entities)}

# Create relation-to-index mapping
unique_relations = set(resampled_df['relation'])
relation_to_idx = {relation: idx for idx, relation in enumerate(unique_relations)}

# Create the graph with the resampled data
graph = create_graph_from_triples(resampled_df, entity_to_idx, relation_to_idx)

# Move graph to the same device as model
graph = graph.to(device)

# Proceed with training as usual, using the resampled data
train_df, test_df = train_test_split(resampled_df, test_size=0.2)

train_dataset = SLDataset(train_df, entity_to_idx)
test_dataset = SLDataset(test_df, entity_to_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Continue with model definition, training loop, etc.



In [ ]:

class_weights = compute_class_weight('balanced', classes=np.unique(resampled_df['relation']), y=resampled_df['relation'])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])  # Gene_NONSL_Gene is the minority class

# 5. Training loop with confusion matrix and relations tracking
n_entities = len(entity_to_idx)
n_relations = len(set(resampled_df['relation']))
n_factors = 4
emb_dim = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SLGNNModel(n_entities, n_relations, n_factors, emb_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)  # Lower learning rate for better convergence

for epoch in range(1, 2):  # Run for 10 epochs instead of just 1
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []

    relations_built = set()

    for gene_a, gene_b, labels in train_loader:
        gene_a = gene_a.to(device)
        gene_b = gene_b.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        logits = model(graph, gene_a, gene_b)

        loss = criterion(logits.squeeze(), labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Track relations
        for idx in range(len(gene_a)):
            edge_id = graph.edge_ids(gene_a[idx], gene_b[idx])  # Updated edge_ids method
            relation = graph.edata['relation'][edge_id].item()
            relations_built.add(relation)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(torch.sigmoid(logits).detach().cpu().numpy())  # Detach before numpy()

    all_preds = np.array(all_preds) > 0.5  # Convert predictions to binary

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Compute Precision, Recall, F1 Score, and ROC-AUC
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    roc_auc = roc_auc_score(all_labels, all_preds)

    # Print or log the results
    print(f"Epoch {epoch}:")
    print(f"Loss: {total_loss / len(train_loader)}")
    print(f"Relations built this epoch: {len(relations_built)}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC-AUC: {roc_auc}")


In [ ]:
# Focal Loss Definition
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
import dgl
from dgl.nn import GATConv
from torch.utils.data import Dataset, DataLoader
from imblearn.over_sampling import RandomOverSampler

# 6. Training loop with confusion matrix and relations tracking
n_entities = len(entity_to_idx)
n_relations = len(set(filtered_df['relation']))
n_factors = 4
emb_dim = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SLGNNModel(n_entities, n_relations, n_factors, emb_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss(reduction='none')(inputs, targets)
        p_t = torch.exp(-BCE_loss)
        loss = self.alpha * (1 - p_t) ** self.gamma * BCE_loss
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

# Use Focal Loss for better class handling
criterion = FocalLoss(alpha=0.25, gamma=2, reduction='mean')

# Continue with the training loop as previously, but now using Focal Loss
for epoch in range(1, 11):  # Run for 10 epochs instead of just 1
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []

    relations_built = set()

    for gene_a, gene_b, labels in train_loader:
        gene_a = gene_a.to(device)
        gene_b = gene_b.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        logits = model(graph, gene_a, gene_b)

        loss = criterion(logits.squeeze(), labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Track relations
        for idx in range(len(gene_a)):
            edge_id = graph.edge_ids(gene_a[idx], gene_b[idx])  # Updated edge_ids method
            relation = graph.edata['relation'][edge_id].item()
            relations_built.add(relation)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(torch.sigmoid(logits).detach().cpu().numpy())  # Detach before numpy()

    all_preds = np.array(all_preds) > 0.5  # Convert predictions to binary

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Compute Precision, Recall, F1 Score, and ROC-AUC
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    roc_auc = roc_auc_score(all_labels, all_preds)

    # Print or log the results
    print(f"Epoch {epoch}:")
    print(f"Loss: {total_loss / len(train_loader)}")
    print(f"Relations built this epoch: {len(relations_built)}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC-AUC: {roc_auc}")


/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \
/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Epoch 1:
Loss: 0.00012826401302318236
Relations built this epoch: 2
Confusion Matrix:
[[      0       0]
 [    126 2894498]]
Precision: 1.0
Recall: 0.9999564710304344
F1 Score: 0.9999782350415141
ROC-AUC: nan


/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Epoch 2:
Loss: 4.474581284917882e-05
Relations built this epoch: 2
Confusion Matrix:
[[      0       0]
 [      6 2894618]]
Precision: 1.0
Recall: 0.9999979271919255
F1 Score: 0.9999989635948886
ROC-AUC: nan


/home/aman24012/miniconda3/envs/aman-24012/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Epoch 3:
Loss: 1.8949330219421798e-05
Relations built this epoch: 2
Confusion Matrix:
[[      0       0]
 [      4 2894620]]
Precision: 1.0
Recall: 0.9999986181279503
F1 Score: 0.9999993090634978
ROC-AUC: nan


In [2]:
Explanation Paths for TP53 → RAB1B: [['TP53', np.str_('TSC1'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('TSC1'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('TSC1'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('TSC1'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('TSC1'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('TSC1'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('TSC1'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('ACVR2A'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('CARM1'), np.str_('CREBBP'), np.str_('RAB1B')], ['TP53', np.str_('CARM1'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('CARM1'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('CARM1'), np.str_('FABP4'), np.str_('RAB1B')], ['TP53', np.str_('CARM1'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('CARM1'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('CARM1'), np.str_('NPR2'), np.str_('RAB1B')], ['TP53', np.str_('CARM1'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('CARM1'), np.str_('HDAC2'), np.str_('RAB1B')], ['TP53', np.str_('CARM1'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('CARM1'), np.str_('SKP2'), np.str_('RAB1B')], ['TP53', np.str_('AKAP17A'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('EIF2AK3'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('SRPK3'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('UBE2E1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('MYL11'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('ABTB3'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('OFCC1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('AEBP2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('CCNH'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('DHX15'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('CCDC171'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('RAD9B'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('RIPOR3'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('APRG1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('POLG'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('FANCA'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('RNF41'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('RAB33B'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('TUBG1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('PSKH2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('STAT1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('DET1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('EXD2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('PRPF19'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('UBE2D1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('GLT8D2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('BFAR'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('SREBF2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('THAP10'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('MYBL2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('UBE2M'), np.str_('RAB1B')], ['TP53', np.str_('EIF2AK2'), np.str_('SPACA7'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PAFAH1B1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('NUBP1'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('VRK1'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('VRK1'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('VRK1'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('VRK1'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('VRK1'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('VRK1'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('VRK1'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('VRK1'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('NAA50'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('INTS3'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('EIF2B2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MRPS9'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('EMC2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('EIF5B'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MRPL19'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('COX5A'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('COQ2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('NDUFB2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MARS2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CDC27'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MTPAP'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('EIF6'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('FAM50A'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('EFR3A'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MAT2A'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CSTF3'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('DHODH'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CCNK'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('CDC16'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('CAPZB'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('POLB'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RNF8'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HNRNPU'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('LYAR'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PCDHGA9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TARDBP'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HSP90AB1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HDAC2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('IARS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PDGFRA'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CCNK'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('KCNMA1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DLG4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HOXC5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CDC16'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MCM6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NVL'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NARS1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SMC4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NCL'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PAM16'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('C1QBP'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ALMS1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TAPT1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('KPNA2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MED7'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DICER1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('AP2B1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('COG6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PARN'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NLRP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DNM1L'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GTPBP8'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CMTR1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ANKRD20A1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ARF4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CYP3A4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAPKAPK5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('INIP'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('IL1B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PSMD11'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPL47'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAP4K5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PMAIP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RFWD3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RABGGTB'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FLT3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('AMH'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GRK6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PKLR'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NSUN2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NDUFB2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SCPEP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UTP6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MTPAP'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ARID1B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DHX15'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SEMG1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ANAPC10'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ATXN3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PTPN14'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ANAPC7'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RBM39'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('VPS39'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PRPF19'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PIWIL4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PRELID1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('THAP10'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TAF6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('IMPDH2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ZNF550'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HUS1B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MUTYH'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPRD1B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MTA2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GTF2I'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MTREX'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UBE3B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CRYBB2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NAA50'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CYLD'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SS18'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MYBL2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FANCA'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EIF2B2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CKS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('AMFR'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SFPQ'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MIA'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('POMC'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PRKCZ'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MDFI'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('AXL'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPL19'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SLX4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('POLDIP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('POT1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPRD2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PIK3R4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RUNX2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('IL12A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PRDX3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FANCL'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DCLRE1A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GLI2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RDM1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PTPRJ'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TOPBP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAT2A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('LIG4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FBN1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PLAA'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FLT1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ERGIC2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PML'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PIK3CD'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPL58'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PRKAB2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PTPN23'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SAE1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EBF3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EXD2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EIF4E'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BARD1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('IREB2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EFTUD2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DHODH'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EIF2AK2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RNASEH2B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DACH1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TNK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HSPA9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FN3K'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UGP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TBC1D1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MLH3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SP3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('IGFBP3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UBE2I'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HSPA5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('LIN52'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UBE2D3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TUBG1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CSTF3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MDM2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NPR2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GML'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('QARS1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DMAP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('USP48'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('KIFC1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NIFK'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BRD2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FBXO9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BAK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EFR3A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CDKL5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FAF2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ITGA6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ACTR5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ANKZF1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PINX1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DFFA'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CANT1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GDF7'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RLF'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GINS1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ZNF223'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('INTS3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UBE4B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TAF5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RAB29'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GRIA1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PAPSS1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('IPO11'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TRUB2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('REV1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FDXR'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SPINK2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FGF3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CAVIN1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PAFAH1B1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('WDR62'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HUWE1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GSC'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ALAD'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RMI1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MDC1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GAK'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PLPP5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ADAMTS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RARS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FOXN4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPA4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TRIM7'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BUB1B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TUBGCP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('WRNIP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAPT'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('H3-3B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('URI1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NNT'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('POLG'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PRPF3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ALYREF'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DHX38'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TERF2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('LARS1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SEM1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ATXN10'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UBE2M'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SON'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BUD13'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPS22'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NFE2L2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DDX50'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CMTR2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FURIN'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CSTF1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GTSE1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GCK'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EME2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TRIM52'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CBR1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BCL2A1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SOX7'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NR2C2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('POLR2L'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MTG2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SF3B3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PSMD3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PLA2G2E'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EMC2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('GATAD2B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('POLR2H'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CDC27'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CCT3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ARID1A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NR1I2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PDS5B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EHMT2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TOP2B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('STYXL1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EFL1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ALDH2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FBH1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PRKD1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CCT2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HKDC1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HK2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SNRK'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EMC6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('IMP3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NUP214'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('COG8'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RBMX2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ABCB7'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ABCG4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PDE3A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EMC4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CPSF4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PRKAG3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RAD51D'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('POLR2I'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('KCND1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CYP1B1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RIPK4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('APAF1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MYL11'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MED19'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NONO'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DNAJB1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('AGTR1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SF3A1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPL9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UHRF2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MARS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CUL4B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CUL5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RCC1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('AURKAIP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('POLD4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MARCHF6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PTCH1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BMAL1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SRD5A1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SRP54'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('AK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('WDR48'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ERCC8'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('KCTD3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ATG10'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EIF6'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SMPD2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ADAT2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('H2AX'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('DET1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EXOSC3'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('HSPD1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('LATS1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RMI2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PATZ1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TRIM21'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EIF3B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PSKH2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('UBE2C'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SCFD1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RNF213'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('ATP6V1G1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NCAPG'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('PFKM'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CASP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('EIF5B'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('TOMM40'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NGDN'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('COQ2'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('NDUFS8'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('CSNK1E'), np.str_('RAB1B')], ['TP53', np.str_('CDK9'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('EHMT2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('EFTUD2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('GTSE1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('OR2B11'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('FABP4'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RIC1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('NR5A1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PTPRJ'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('FOXN4'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('VPS54'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('DET1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('APRG1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TRIM46'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('IL12A'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('LRRC10'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RPS6KA2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ABCB7'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RSRP1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('CRY2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ALAD'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('POLR2L'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('CYP1B1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('NR1I2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TRIM52'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ITGA6'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PAPSS1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('C1QBP'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TMEM61'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PDE3A'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('UBA6'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('ATG7'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('HDAC2'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('PDGFRA'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('MUC16'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('FLT3'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('PML'), np.str_('RAB1B')], ['TP53', np.str_('CHEK2'), np.str_('MDM4'), np.str_('RAB1B')], ['TP53', np.str_('FASTKD2'), np.str_('CNOT8'), np.str_('RAB1B')], ['TP53', np.str_('LMLN'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('LMLN'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('GTSE1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CYP1B1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SLX4'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MAPK10'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PRKAG3'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('KCND1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('IL12A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NCOR1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PTPA'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RIPK4'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RB1CC1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('FABP4'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RTEL1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MIA'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PRKAR1A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('FANCL'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NAGK'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PLA2G2E'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('HSP90AB1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SEMG1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PBX1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('HOXA13'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PTCH1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('DMAP1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RAD51D'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('GCK'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('GSC'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MDH1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('GRIA1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('EHMT2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PAM16'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ORC5'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('HK2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('LAPTM5'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CDKN1A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('BCAR1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('LIG4'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PRKD1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('DNAJB1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('HSPA5'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MTPAP'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ENPP1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('EXOSC3'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SEPTIN1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('FN3K'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CASP6'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PFKM'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RPS6KA2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NARS1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CHMP1A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('H2AX'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('FANCA'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ABCB7'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('BMAL1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SRD5A1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('STYXL1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SCPEP1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('IL2RG'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MRPL35'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('APAF1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('BRD2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CANT1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('AMPH'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ARID1A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CKS2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ATG7'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ITGA6'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NLRP2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('NR5A1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KB1'), np.str_('CDC27'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('C6orf136'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('MEP1A'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('KBTBD3'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('TOMM6'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('UNKL'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('RGS22'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('PSKH2'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('RIPOR3'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('IKZF2'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('CYP3A4'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('OR2B11'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('FAM76B'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('PLEKHS1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('GIMAP2'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('FOXR1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('SPATA6L'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('SEPTIN1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('ZNF705EP'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('CALML4'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('PDCD11'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('SH2D3A'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('ST7'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('R3HDM2'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('OAF'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('TMEM248'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('STARD6'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('TMEM61'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('TOGARAM1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('FAM167A'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('FREM1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('ABTB3'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('LRRN1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('LRRN4CL'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('RSRP1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('THAP10'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('ADAMTSL5'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('ZNF512'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('CPNE5'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('XAGE3'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('NADK'), np.str_('SPACA7'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('PRKAR2A'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('DENND4A'), np.str_('DENND4B'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SLC43A3'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('ZNF521'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('PTCD1'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('YWHAG'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('YWHAG'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('YWHAG'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('YWHAG'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('IL2RG'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('THADA'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('ITGB7'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('ITGB7'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('ITGB7'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('ITGB7'), np.str_('POMC'), np.str_('RAB1B')], ['TP53', np.str_('ITGB7'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('ITGB7'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('GNGT1'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('DKC1'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('BCL2L14'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2L14'), np.str_('HRK'), np.str_('RAB1B')], ['TP53', np.str_('BCL2L14'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2L14'), np.str_('BNIP3'), np.str_('RAB1B')], ['TP53', np.str_('BCL2L14'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('BCL2L14'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('KSR1'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('PLEKHS1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('QARS1'), np.str_('RAB1B')], ['TP53', np.str_('EIF2B3'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('MELK'), np.str_('SNRK'), np.str_('RAB1B')], ['TP53', np.str_('MELK'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('MELK'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('MELK'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('OXSR1'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('OXSR1'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('OXSR1'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('OXSR1'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('OXSR1'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('OXSR1'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('OXSR1'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('CTBS'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('TRIM52'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('CCNH'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('RPS6KA6'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('NOTCH1'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('NOL8'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('VPS51'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('NEK11'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('ATG7'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('WDR62'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('TBC1D1'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('ATG12'), np.str_('RAB1B')], ['TP53', np.str_('ABL2'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('PBX1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('SEPTIN1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('RPS6KA2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('IL12A'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('SLX4'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('TSC2'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RNF213'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PKLR'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EHMT2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PRKAG3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('POMC'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ATP6V1G1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ARID1B'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NR1I2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('HK2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CRYBB2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ATXN10'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('VPS39'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ARID1A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('FURIN'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CMTR2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ALDH2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('UBE4B'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NAGK'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PTCH1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CDC27'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SNRK'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('AK1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SECISBP2L'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ENPP1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SLX4'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SCPEP1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EIF2B2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CRY2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('IL12A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MTPAP'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PLA2G2E'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('AXL'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RPRD2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('POLD4'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PRKAR1A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('USP8'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PRPF19'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PI4KB'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('COG8'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EXOSC3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MAPK10'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('H2AX'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('DICER1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RIC1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RPS6KA2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CASP6'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('UBE3A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('COG6'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('DNAJB1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ATM'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EMC6'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CANT1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('COQ2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('VPS54'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PBX1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PIKFYVE'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SMPD2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TYRO3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CHUK'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PDE3A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('HSP90AB1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ASCC3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('FLT1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CSTF3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CDKN1A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('GTSE1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ADAMTS2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NLRP2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ACTR5'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EMC2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('BRD2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('EIF5B'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('KCND1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CASP1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('STYXL1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CCNK'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RIPK4'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('DMAP1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('AURKAIP1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PRKD1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PAM16'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('HIPK1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('APAF1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ABCB7'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('FABP4'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ALAD'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('RPS6KA4'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('HSF1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('GRIA1'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('ITGA6'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('COX5A'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('PRKCZ'), np.str_('PFKM'), np.str_('RAB1B')], ['TP53', np.str_('TYMS'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('TYMS'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('TYMS'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('TYMS'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('TYMS'), np.str_('BARD1'), np.str_('RAB1B')], ['TP53', np.str_('TYMS'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('TYMS'), np.str_('TYRO3'), np.str_('RAB1B')], ['TP53', np.str_('MT2A'), np.str_('CAVIN3'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('ADAT2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('BUB1B'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('ANKZF1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('EIF2B2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('DICER1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('COQ2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('NDUFB2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPL19'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('NAA50'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPS9'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('ASCC3'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('EIF6'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPL35'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('IMPDH2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('EMC2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('EFR3A'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('HCCS'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('EIF5B'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('CSTF3'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('EMC6'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('CDC27'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('ACTR10'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K7'), np.str_('PIK3CD'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K7'), np.str_('TACR1'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('RPL32'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('SKP2'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('POLD4'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('PDGFRA'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('RPL13A'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('PTPA'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('USP21'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('PML'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('PARP1'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('ELP5'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('PELO'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('MARK2'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('MARK2'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('TLK2'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('TLK2'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('TLK2'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PML'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('WDR62'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('UBE2D1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('H2AX'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RTEL1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('FURIN'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ERCC6L'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PAPOLA'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('GLI2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NELFE'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CANT1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PTPA'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TDRD10'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MTA2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PDE3A'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CUL4B'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('FAM50A'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SON'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NCOR1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('EIF4E'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NAIP'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NR1I2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('STAT1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PAPSS1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('LIG4'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RIPK4'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ENPP1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MAPKAPK5'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MDM4'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('EMC2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RBL2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ERGIC2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ATG12'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('GSC'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('DAXX'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('POLG'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SEM1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('DNAJB1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PRKAG3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MDC1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('APAF1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('DFFA'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ATP6V1G1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SRP54'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CCNH'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CARD9'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('COX5A'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RCC1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('AURKAIP1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('KCND1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CDX2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CCNK'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NFE2L2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PFKM'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PALMD'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('FANCL'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ADAMTS2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NAGK'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RNF8'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('STYXL1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PARN'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CPSF4'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ATXN3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ALYREF'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NONO'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CYP1B1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MDH1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RPS6KA2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CASP6'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TCF20'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ABCB7'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NARS1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('AK1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ZFHX3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CDK2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ASCC3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('VPS54'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('AGTR1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ITGA6'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NNT'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NR5A1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('AMFR'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NDUFS8'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RPRD2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PSMD11'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('USP8'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PDS5B'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ATG3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CSTF1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('H4C11'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('EIF2AK3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CKS2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('AMPH'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('HSF1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PCGF5'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PRKAR1A'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RXYLT1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('AP2B1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('EXOSC10'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NUP214'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('DICER1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('BMAL1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('HSPA9'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CRY2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('HSPA5'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('H3-3B'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('LARS1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SRD5A1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CHUK'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('TOPBP1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('CUL5'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('MRPL35'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SKP2'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('SECISBP2L'), np.str_('RAB1B')], ['TP53', np.str_('BRD2'), np.str_('FLT1'), np.str_('RAB1B')], ['TP53', np.str_('SRPK1'), np.str_('SRPK3'), np.str_('RAB1B')], ['TP53', np.str_('SRPK1'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('IRAK1'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('IRAK1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('PHKG2'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RPS6KA4'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('EIF2B2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('KCND1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('YJU2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('GINS1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('UBE3A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ADAMTS2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PLA2G2E'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CRYBB2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ERGIC2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PAM16'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PDE3A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PML'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('AXL'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('GRK6'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CPSF4'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ASCC3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('IL2RG'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NR1I2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NPTN'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RPL13A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CRY2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RIPK4'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('FABP4'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('BMAL1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PIK3CD'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('POMC'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TM9SF2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PBX1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NFE2L2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('BRD2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('HIPK1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('AMHR2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NPR2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SMPD2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NAGK'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TTN'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MDM4'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ATP6V1G1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ARF4'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PAFAH1B1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MAP4K5'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('COG6'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CKS2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TNK1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('FBN1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('UGCG'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('C1QBP'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MDH1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('HK2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('BUB1B'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NEK11'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CSNK1E'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('FLT1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CCT2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('UGP2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CDK2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SECISBP2L'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('GCK'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('AURKAIP1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ESPL1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('LARS1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('STYXL1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ATM'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('AK1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MDM2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PTPA'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SNRK'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('IGFBP3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CNOT8'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PRKD1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PTPRJ'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CHUK'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('FURIN'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('EIF4E'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PAPSS1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ERCC6L'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RNF213'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('EFR3A'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('GTSE1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RTEL1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('EHMT2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('H2AX'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('FN3K'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('VPS54'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('RXYLT1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('ALAD'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('HDAC2'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('PDPK1'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('RSKR'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('PRKCI'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('VIM'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('CELF5'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('PGM1'), np.str_('STK4'), np.str_('RAB1B')], ['TP53', np.str_('RHAG'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('RHAG'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('RHAG'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('RHAG'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('RHAG'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('RHAG'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('RHAG'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('RHAG'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('RHAG'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('RHAG'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FLYWCH1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('KBTBD3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ERCC8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PKLR'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BRD2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FGF3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ABCB7'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('STYXL1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PAFAH1B1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('IREB2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('WDR62'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ATG3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('KIF18A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('USP8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PBX1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ANKZF1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SON'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BRD8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ITPRID1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DNM1L'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SOX7'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SAFB2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TOPBP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DLG2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPL2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PTPRJ'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PAPSS1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ATG7'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HSPA5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TEC'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DFFA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPS6KA2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AK1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CNIH1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PTPN23'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('LARS1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DDX50'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PANK3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PCGF6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AGTR1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GTF2I'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AXL'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CUL4B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MDM4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UBE3B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SECISBP2L'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FAM50A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CYP3A4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RING1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RTEL1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FLT1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('POLG'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ERCC6L'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('H2AX'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SMC4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PLAA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DMAP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CCNH'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BAK1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PARN'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NNT'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GRK6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FDXR'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('IFT57'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FANCL'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('COL25A1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('WDR91'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SCARF1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('LATS1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CDX2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CDC27'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EMC2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HSPA9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('POLR2D'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CRYBB2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NLRP2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RABGGTB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPL9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NARS1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPL35'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('VPS54'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FURIN'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GRK4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MYBL2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TSPYL5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UBE4B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MDM2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NAPA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CSNK2A1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CREG1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DGKK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TNK1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TBC1D1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ATG4A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RXYLT1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TRPM6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EIF6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('COG6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EFTUD2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CCNK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FBN1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ATG10'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ALYREF'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ZNF638'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ATG12'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GAK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CANT1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ABCG4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NUP214'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MTG2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CMTR2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NVL'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TM9SF2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ALDH2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PSMD3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('IL12A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ADAMTS2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPRD2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UBXN1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EIF5B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HCCS'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MPO'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ZFHX3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DHODH'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NR5A1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HTR1E'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PRDM4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SNX8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MED7'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CHUK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HSP90AB1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FBH1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EHMT2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UBE3A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPL19'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CRY2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ZNF517'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HK2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CCNA2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RAB6A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UGP2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CYP2J2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PPL'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EMC6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HIPK1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GADD45A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NCOR1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PSMD11'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RCC1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ITGA6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UBE2C'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('XAGE3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EFR3A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AMPH'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PRKAB2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GDF7'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DCLRE1A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('H4C11'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RIPK4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CYP1B1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EBF3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('IL1B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SCPEP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MTPAP'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ACTR5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SELENOS'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('STAT1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GRAP2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('POMC'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PBK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ANKFY1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ADAT2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CDKL5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('VPS39'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PI4KB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EIF2AK2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FN3K'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ESPL1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('METTL14'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CCT2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AOC3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ARID1B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CSTF3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ATXN10'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('H2AC1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UFL1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PTPA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DLG1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BRMS1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NFE2L2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PRKD1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GLI2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RNF213'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CSTF1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CDC16'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SMPD2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PIK3CD'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('APAF1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DNAJB1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ERCC4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SRP54'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RNF8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ATP6V1G1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BARD1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MDC1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SLX4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('POLD4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RMI2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PAM16'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NAGK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('URI1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NPR2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('KCTD3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PFKM'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CCDC14'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FABP4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ANKRD26'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PDE3A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GCK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CES2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SAE1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SSH1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CUL5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RUNX2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CPSF4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ASCC3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UGCG'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CCT3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TAPT1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MTA2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AEBP2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPL13A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PDPK1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RDM1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PPP6R3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ZFP36'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HOXC11'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('COQ2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAPKAPK5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CD2BP2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PRKAR1A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('LIG4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CSNK1E'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAT2A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('VWCE'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GRIA1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GSC'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NF2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TRIM8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RGS22'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('IL2RG'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RIC1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MARCHF6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TYRO3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GML'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PLA2G2E'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TTN'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ARFRP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ATXN3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GTSE1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AP2B1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PTCH1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PIKFYVE'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('OR4F6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GINS1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CDK2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CASP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MIA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ZNF74'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('QARS1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UBE2K'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('POLB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('POLR2I'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('POLDIP3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NAA50'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SRPK3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UBXN7'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('FANCA'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TTLL3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RNF41'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('WDR48'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SRP9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RAB9A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NONO'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('EXOSC3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PDS5B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GTPBP8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NDUFS8'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('COP1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPS9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CHMP1A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RFX4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ALMS1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ERGIC2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAP4K5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CBR1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('AMHR2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PLIN2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MAPK10'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('KCND1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PNCK'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('PLK2'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('BMAL1'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('VHL'), np.str_('GALNT10'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('WEE2'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('POLR2D'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('UBE2D1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('WDR62'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('RBL2'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('ERCC8'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('TUBG1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('TBC1D1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PSMD3'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('NDUFS8'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('UBE2V2'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('SRP54'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('GRIA1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('CPSF4'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('DICER1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('STAT1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('EIF2A'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('AGTR1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('CSTF1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('RB1CC1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PRPF19'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('NONO'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('DHODH'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('EBF3'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('GLI2'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('FDXR'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('ENPP1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('NAIP'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('MTA2'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('VPS39'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('DDX27'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('AMH'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('DDX52'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('ALMS1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('DDX50'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('EIF2S3'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('GRSF1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('WEE1'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('VMP1'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('VMP1'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ENPP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RNF8'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PBX1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RBL2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('SEMG1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RPS6KA4'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('GSC'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CSTF1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MTA2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('POLR2D'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('EHMT2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('COX5A'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('NUP214'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ERCC8'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CCNH'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('GLI2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RPL12'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RCC1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K4'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('KDM5B'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('TUBA1B'), np.str_('GINS1'), np.str_('RAB1B')], ['TP53', np.str_('TUBA1B'), np.str_('NCAPG'), np.str_('RAB1B')], ['TP53', np.str_('LIN7C'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('ULK4'), np.str_('USP8'), np.str_('RAB1B')], ['TP53', np.str_('TICRR'), np.str_('TOPBP1'), np.str_('RAB1B')], ['TP53', np.str_('PAG1'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('RPL28'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('BUB1'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('BUB1'), np.str_('CYP3A4'), np.str_('RAB1B')], ['TP53', np.str_('BUB1'), np.str_('CYP1B1'), np.str_('RAB1B')], ['TP53', np.str_('BUB1'), np.str_('NR1I2'), np.str_('RAB1B')], ['TP53', np.str_('BUB1'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('BUB1'), np.str_('COL14A1'), np.str_('RAB1B')], ['TP53', np.str_('BUB1'), np.str_('MAPT'), np.str_('RAB1B')], ['TP53', np.str_('BUB1'), np.str_('SMC4'), np.str_('RAB1B')], ['TP53', np.str_('UCK2'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('UCK2'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('CTCF'), np.str_('TERF2'), np.str_('RAB1B')], ['TP53', np.str_('CTCF'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RPL13A'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PIK3R4'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RB1CC1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SH3GLB1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('NDUFB2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('STX8'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('ATG7'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PDGFRA'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('POLD4'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('NAA50'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PAFAH1B1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('SREBF2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('TRIM21'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RAB33B'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RAB12'), np.str_('RAB1B')], ['TP53', np.str_('MTOR'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('KHK'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('IL2RG'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('RORA'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('DICER1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('COQ2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('EMC4'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MAT2A'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('NAA50'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MTPAP'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('IMPDH2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('EIF5B'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('FAM50A'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('HCCS'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('NDUFB2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('DHODH'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('EIF6'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('EFR3A'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('EIF2B2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MARS2'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('MRPL35'), np.str_('RAB1B')], ['TP53', np.str_('CEP192'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('HSPD1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('DHODH'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MCM6'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('GML'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TMEM86B'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CDKN1A'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CALML4'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('POLDIP2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TMEM61'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('KCNMA1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPS33'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MYBL2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPS9'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CCNK'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('KBTBD3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('EXOSC10'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('DHX15'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PIGV'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('EXOSC3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('URI1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('UFL1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MED7'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NR2C2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPL2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RMI1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RLF'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('H4C11'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PFKM'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('FOXN4'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ERCC6L'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SEPTIN1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CDC16'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('EFL1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PRPF19'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('DICER1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NECAP1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SREBF2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TUBG1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RBM39'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('POLR2H'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('EIF6'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MPO'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RGP1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MUTYH'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('BAK1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NCOR1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('LRRN4CL'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MTG2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PDS5B'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('IL22RA2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('FREM1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('UHRF2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CYP3A4'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SPINK2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CCT3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CCNA2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('DNAJC5G'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CCT2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('BUB1B'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RGS22'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPL58'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('UBE2K'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('GTF2I'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PINX1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CCDC82'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TOPBP1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('C1QBP'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('EXD2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('EIF2AK2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('POLDIP3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ANAPC10'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('BARD1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NELFE'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SMC4'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('UBE2E1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RNF8'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MAPKAPK5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('EME2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ATM'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('FAM167A'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TOMM6'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('POLR2D'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('DHX38'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('HOXC5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('INIP'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('BFAR'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RMI2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('HSP90AB1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('AEBP2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('XAGE3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PLPP5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CSNK2A1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('HSPA9'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('POLR2L'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ZNF550'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ILF2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPL9'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPL19'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('FANCA'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MDC1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ANAPC7'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('POLR2I'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('DCLRE1A'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('SAE1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RFWD3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PSKH2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('UBE2M'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CDC27'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('HSPA5'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('HNRNPU'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('TRUB2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('INTS3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('UBE3B'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('MRPL35'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ALMS1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NFE2L2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('NGDN'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ASCC3'), np.str_('RAB1B')], ['TP53', np.str_('UBE2I'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('IL12A'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('NLRP2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('BRD2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('EIF4E'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('EHMT2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('FN3K'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CCT2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CRYBB2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('NR5A1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('POMC'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PML'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MDM4'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('NAGK'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PKLR'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CDKN1A'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('POLD4'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PLA2G2E'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PRKAR1A'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PTCH1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('SLX4'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('FABP4'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('AXL'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ATM'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('RPS6KA4'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ARID1A'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CRY2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ZNF296'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('IGFBP3'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PRKCZ'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ITGA6'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CDK2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('CANT1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PTPRJ'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('ACVR1'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('PRKCSH'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('GYPC'), np.str_('MYBL2'), np.str_('RAB1B')], ['TP53', np.str_('NSD1'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('NSD1'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('GNE'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('UBR4'), np.str_('ARFGAP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('TTN'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('MAP2K1'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K13'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('PDGFRA'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('HDAC2'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('FLT3'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('BRAF'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('KIF2C'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('KIF2C'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('KIF2C'), np.str_('MYBL2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('INTS3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ADAMTS2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PRKD1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TLK1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('GCK'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RNF213'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PIK3CD'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SEPTIN1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('HCCS'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('IL12A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CBR1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('AURKAIP1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ATP6V1G1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MAPT'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('BAK1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PML'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('AMHR2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CDK4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('NPR2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ATM'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ITGA6'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RIC1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SECISBP2L'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ALAD'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ABCB7'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('AOC3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CRYBB2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('EMC4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('FURIN'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('FN3K'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ALDH2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CSNK2A1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MDM2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('GLI2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RABGGTB'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('COQ2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('KCND1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('HSF1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PDPK1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('GRK6'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PTCH1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PDGFRA'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CANT1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CHUK'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('GADD45A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CYP3A4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('POLD4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('AXL'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('POMC'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('BUB1B'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('IGFBP3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SNRK'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ARID1A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RIPK4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('EHMT2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PRKCZ'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('NR1I2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PRKAG3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('FLT1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RPS6KA4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('NLRP2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PFKM'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('NAGK'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('HIPK1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SMPD2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('EIF4E'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RPS6KA2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('EXOSC3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('EFR3A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TYRO3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('IL2RG'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PLA2G2E'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('BRD2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TNK1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('FBH1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('YJU2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ASCC3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('AK1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('VPS54'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('GINS1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SLX4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('FBN1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('UBE3A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CKS2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PAFAH1B1'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MAP4K5'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('CREBBP'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('PDE3A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('MAPK10'), np.str_('RAB1B')], ['TP53', np.str_('MST1R'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('ERN1'), np.str_('ERN2'), np.str_('RAB1B')], ['TP53', np.str_('SPAG8'), np.str_('POU6F2'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('FOXN4'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('INIP'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('GCAT'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('BUD13'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('NFE2L2'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('POLG'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('SFXN3'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('EFTUD2'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('NELFE'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('HK2'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('UTP6'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('TRIM52'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('PRELID1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('RNASEH2B'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('NNT'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('HOXC5'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('CCNH'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('POLR2L'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('TUBG1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('FDXR'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('TOMM40'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('HIBCH'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('MTG2'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('DET1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('PLK2'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('NEK2'), np.str_('MUTYH'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('MAST3'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('MAST2'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('UBE2D1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('ATG7'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('HSF1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('POLR2D'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('RBL2'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('XRCC6'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('HIGD2A'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('MST1R'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('UBE2H'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('PDGFRA'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('MET'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('SCYL2'), np.str_('HSPD1'), np.str_('RAB1B')], ['TP53', np.str_('SCYL2'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('NFYB'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('NFYB'), np.str_('RAPH1'), np.str_('RAB1B')], ['TP53', np.str_('NFYB'), np.str_('ZDHHC17'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('TRIM7'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('MRPS33'), np.str_('RAB1B')], ['TP53', np.str_('PDIK1L'), np.str_('POLR2L'), np.str_('RAB1B')], ['TP53', np.str_('CTSC'), np.str_('CTSZ'), np.str_('RAB1B')], ['TP53', np.str_('DDX21'), np.str_('DDX50'), np.str_('RAB1B')], ['TP53', np.str_('DDX21'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('DDX21'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('DDR2'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('MICOS10'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('MICOS10'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('MICOS10'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('MICOS10'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('MICOS10'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('MICOS10'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('MICOS10'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('MICOS10'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('MICOS10'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('MICOS10'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('SEMG1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('LAPTM5'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('HSF1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('DNAJB1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('PKLR'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('EHMT2'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('ERBB3'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('PHF5A'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('LARS1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('NDUFB2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('HCCS'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MAT2A'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPS9'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('INTS3'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('IMPDH2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPL19'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MARS2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('FARSB'), np.str_('HSPA5'), np.str_('RAB1B')], ['TP53', np.str_('PLOD2'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('PLOD2'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('PLOD2'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('STK38'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('STK38'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('ASF1A'), np.str_('KPNA5'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PLK2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MTR'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SFXN3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DLG4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PSKH2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('EXD2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MIA'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DET1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TUBGCP2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RPS6KA2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('FBXO9'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('POLR2D'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('BPIFA3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('GLI2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CPSF4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ENPP1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NDUFS8'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('OR10G4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ANKRD20A1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CUL4B'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NONO'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PDS5B'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ORC5'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CCNH'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ATG7'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TBC1D1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RTL6'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('QARS1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PBX1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('LIG4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TRUB2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('GCAT'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('GSC'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('R3HDM2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PRPF19'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SRP54'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('UFL1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('OR1N2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NELFE'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('UBE2V2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ANAPC10'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('EMC2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DDX52'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RLF'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('BUD13'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RCC1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('APRG1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RNF8'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MKI67'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DENND5B'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MYBL2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TRIM52'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PBOV1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('UBE2D1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('APAF1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ATG10'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('AMFR'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DAXX'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('C1QBP'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('EFTUD2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('WDR62'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RTEL1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('EMC4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('FANCA'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ALYREF'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('INIP'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RIC1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DHX15'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RIPOR3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('KCNMA1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('XAGE3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DMAP1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MUTYH'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('POLG'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('GTF2I'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NGDN'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DFFA'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DHODH'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PARN'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PAPSS1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('UBE2M'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SPINK2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DICER1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TOMM6'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ERGIC2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CD2BP2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('EIF2AK2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CSTF1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('HSF1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TUBG2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RABGGTB'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CCNK'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('COX5A'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('EIF2AK3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('HKDC1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CSTF3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('EIF6'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('AGTR1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ERCC6L'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RBL2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SEMG1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TCF20'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MRPL58'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SRD5A1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PSMD3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TUBG1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SLX4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SEM1'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('HSPA9'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PRKAG3'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('PIWIL4'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('CKS2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('ABCB7'), np.str_('RAB1B')], ['TP53', np.str_('PLK1'), np.str_('FIGN'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('SNAPC1'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('PAFAH1B1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MRPL19'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('MED17'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('USP7'), np.str_('ZNF263'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('SRPK3'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('EME2'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('GCK'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('ATM'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('SEPTIN1'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('IL2RG'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('SRPK2'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('LAPTM5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ARID1B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TRPM6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SRD5A1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('GADD45A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MYBL2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RIPK4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PKLR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MRPS9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('HSP90AB1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CCDC171'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('EIF6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CDC16'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TNK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('VPS39'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CCT2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MDM4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('DAXX'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PRKAG3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CRYBB2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PML'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PLA2G2E'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ERCC6L'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CSTF3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('HIPK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('COX5A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PIKFYVE'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MAPKAPK5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FN3K'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FAM50A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ESPL1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('POMC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CCNK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MDC1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PRPF19'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('LSS'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('DNAJB1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('HSF1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('GDF7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NAGK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('UBE2C'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ZNF517'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('COG8'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('VPS54'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RNF213'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('POLD4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PRKCZ'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('DMAP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('AXL'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ATP6V1G1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NFE2L2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PTPA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('YJU2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MAT2A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('WDR62'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MDH1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('EMC4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('DICER1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('BUD13'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ASCC3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SNRK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MIA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CANT1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CPSF4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CKS2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('USP8'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PBX1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ITGA6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PAM16'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('IL12A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SLX4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TM9SF2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CRY2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RPS6KA4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RXYLT1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FLT1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RPL13A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SMPD2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CARD9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NDUFB2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MTPAP'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PSKH2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NCOR1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PDE3A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CDC27'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FANCA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('INTS3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CNOT8'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('AURKAIP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TYRO3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('EMC2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NR1I2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NAA50'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CYP1B1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PAPOLA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('LARS1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CHUK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SCPEP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ARID1A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ADAT2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('HK2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('UGP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CDX2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('EXOSC3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('GTF2I'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('EIF4E'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('EIF2AK3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MARS2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('GLI2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NR5A1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MAPK10'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ARFRP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FABP4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('EHMT2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('FURIN'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('DHODH'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('URI1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CASP6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('POLR2I'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('GSC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('GCK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ALDH2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('IGFBP3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ERGIC2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SEM1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CCNA2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ATM'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RIC1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SEPTIN1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TOPBP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CDKN1A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('NLRP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CES2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PARN'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('PI4KB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('EIF2B2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('CDK2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('RPRD2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1E'), np.str_('IL2RG'), np.str_('RAB1B')], ['TP53', np.str_('ULK2'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('ULK2'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('ULK2'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('ULK2'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('ULK2'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('ULK2'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('ULK2'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('ULK2'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('GPATCH2'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1G3'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1G3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1G3'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1G3'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1G3'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1G3'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1G3'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1G3'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK1G3'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FLT3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('EFR3A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MAP4K5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NLRP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FLT1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPS6KA4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('DNAJC5G'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('YJU2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('UGP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('COG6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('URI1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('HK2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('AMHR2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('UBE3A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PDE3A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('STAT1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RMI2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ALAD'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('UBE2M'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MDM4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('HSPA9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('DCLRE1A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CANT1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PIGV'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ASCC3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MPO'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NPR2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NDUFS8'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ATM'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TTN'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SEM1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ABCB7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SREBF2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('IL12A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TFCP2L1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PIK3CD'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('AK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('UBE2E1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('POLD4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TMEM86B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('EMC6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SEPTIN1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CCNA2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('H2AX'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PAM16'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('GSC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('GINS1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('IL22RA2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CHUK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MAPK10'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('GTSE1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BAK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('HIPK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TNK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PRKCZ'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NR1I2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FN3K'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('XAGE3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CUL5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('UBE2U'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MIA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NAGK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('POLB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TMEM248'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('GCK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('INTS3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TMEM61'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('GRK6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SAE1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ILF2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPS6KA2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PAFAH1B1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BUB1B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PLK2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PRPF19'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('EIF2B2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('APRG1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PFKM'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FBN1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('GML'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MYL11'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ADAMTS2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('EME2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SNRK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MYBL2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('POMC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CDKL5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ERCC6L'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('DET1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SCPEP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FAM167A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BRD2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('POLG'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SMC4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PRKD1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SMPD2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TTLL3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('DNAJB1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CDK2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PRKAG3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('DDX42'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RIC1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PBX1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPL13A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TYRO3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('VPS54'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('AURKAIP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NR5A1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PAPSS1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PTPA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CARD9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CSNK1E'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ATP6V1G1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CYP3A4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CCT2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('AXL'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('KCND1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('UBE2D1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('DENND5B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PLA2G2E'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FABP4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('IL2RG'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('EIF2AK2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PKLR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MDM2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FURIN'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ALDH2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PRKAR1A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RNF213'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RAD9B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('GADD45A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('RTL6'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('PTPRJ'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('UBE2K'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('ARID1A'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('SECISBP2L'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('CSNK2A1'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('ZBTB16'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RB1CC1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('EIF2B2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('CDK4'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('GABARAPL1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('COX5A'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MTPAP'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('DHODH'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('HSPA5'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('EMC6'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MARS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('HCCS'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('CSTF3'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('FAM50A'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPL35'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('EMC4'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('GABARAP'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('NAA50'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MAP1LC3C'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('NDUFB2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('IMPDH2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPL19'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('STIL'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('DICER1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PIK3R4'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('INTS3'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SNRPA1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('CDK2'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('MRPS9'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('CDK1'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('GFPT2'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('GFPT2'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('GFPT2'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('HDAC4'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('NIM1K'), np.str_('SNRK'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('EPHA1'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('COX5A'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ERGIC2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ABCG4'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ATXN3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CUL4B'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CCDC14'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('EMC2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('EXOSC3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PAPSS1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CPSF4'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('HNRNPU'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('BCL2A1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('EIF6'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('DLG4'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('GAK'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CD2BP2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CNIH1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RBL2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('EXD2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CKS2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('LAPTM5'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SEM1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TRUB2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('GLI2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TUBGCP2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ABCB7'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('OR10G4'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('GTF2I'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('VPS39'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RCC1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('GSC'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ATG10'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('WDR62'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MARCHF6'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ALYREF'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MTA2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SCPEP1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PDS5B'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CCNH'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SKP2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ENPP1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('POLR2D'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CSTF3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TCF20'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RUNX2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CSTF1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ORC5'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('POLR2L'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('UBE2D1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('COG8'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('NFIX'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('AGTR1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CARD9'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CCNK'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SEMG1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PARN'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('EBF3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PRPF19'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CMTR2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MAPKAPK5'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('DHODH'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ATG12'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('EIF3B'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TUBG1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SCARF1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('HSPA9'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('FGF3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MRPL58'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('KIF18A'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('EMC4'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PRKAB2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('HSF1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('ATG7'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('EFTUD2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('NDUFS8'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MARS2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('GDF7'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('RIC1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('RB1'), np.str_('QARS1'), np.str_('RAB1B')], ['TP53', np.str_('KMT2A'), np.str_('TOPBP1'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('STK35'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('MED6'), np.str_('PAPOLA'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('UTP6'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('EIF6'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CDC27'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RPS6KA4'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('DHX38'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('BARD1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CCNH'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MRPL58'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TUBG2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CDKN1A'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('DDX52'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('DDX27'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('POLR2I'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('EXD2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('GCK'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PDIK1L'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CBR1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('H2AX'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('INIP'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('UHRF2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('EXOSC10'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('DCLRE1A'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MRPS9'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('EFTUD2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('HKDC1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('HK2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TRIM52'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CCNA2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CYP3A4'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('DET1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PFKM'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('IARS2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ATM'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TUBG1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MDC1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CCT2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RPA4'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RNF8'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TOPBP1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MRPS33'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MUTYH'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('POLB'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('C1QBP'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('NGDN'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ANAPC10'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RFWD3'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('BUB1B'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('NIFK'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PINX1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('POLR2L'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TOP3A'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('UBE2M'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RMI1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('EME2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('NAA50'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TRIM7'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('FANCL'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SMC4'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('GTF2I'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('IMP3'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('POLDIP2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('POLD4'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('TOP2B'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('MRPL19'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('POLR2D'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('SLX4'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('CDK19'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('RPL13'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('MAPKAPK2'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('MAPKAPK2'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('MAPKAPK2'), np.str_('PRKCZ'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('EIF5B'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MAT2A'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MRPL19'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('EMC2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MARS2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('IMPDH2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MRPS9'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('FAM50A'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('EMC4'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('NDUFB2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('HCCS'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MTPAP'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('INTS3'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('EIF4G2'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('HNF4G'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('GLUD2'), np.str_('MTREX'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPL15'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GAK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TUBG1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SH3GLB1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MCCD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ELK4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FBXL20'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COCH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LRRC42'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NPTN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR1B1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TSNAX'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SFTPD'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCGN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PAX4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SHOC2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SEZ6L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TRIM8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DAZ4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAPKBP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RFWD3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BUD13'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DLG4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SH3BP5L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCN2B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KCNK15'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRPF19'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PON2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDKL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COPS9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ESPL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FOXI1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ALK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM91A1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL48'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C5orf63'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MACIR'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RABL6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CNIH1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PNCK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DUSP18'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PIWIL4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARFRP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TRIM63'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PAPOLA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CMTM3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CSTF3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SMC4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ERAL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FOXR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RIOK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PGBD4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KIN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GINS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RHOBTB3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SERPINA2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SOS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BRSK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NAA30'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATM'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ENPP6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TCF21'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TCF19'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MINPP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF607'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CSNK1D'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF665'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('REPS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KCNC3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('APLNR'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRELID1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LSG1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('XPOT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARSB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FDCSP'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GLMP'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NAT2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GADD45A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ALAD'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GARIN4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM217B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RANBP3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('JPH3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NR2C2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ADAMTS12'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GDF7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNFT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COL14A1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TIFA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CPA2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ITGA6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SPTLC3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MUC17'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF398'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CASP5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PJVK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DPH5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RALGAPA2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATXN10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LAMTOR4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NSUN6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BRMS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BRICD5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SDCCAG8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VIPR2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IFI6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCT3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MICB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAX2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TIAL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DCLRE1A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HEXD'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NPR2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPS23'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SH2D2A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TNFSF12'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE3B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AXL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('H2AC1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TERF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM241A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RRAS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PLEKHO2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPP1R3F'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AS3MT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ALMS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RHPN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IL17RD'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ANOS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GALNT10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM43'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PFKM'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ORC5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CNTN5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE2W'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VASH1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SNX9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ALDH2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RXYLT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BRIX1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZC2HC1C'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('WRNIP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RABGAP1L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SERPINB8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KBTBD12'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FGL2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FBH1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IREB2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FLT3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GLT8D2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('METTL14'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNF24'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TFCP2L1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EBF3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAGOH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNASEH1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IFI35'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TRIM35'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PI4KB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IL17A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DHX37'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COPS8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCDHB7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CUL5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CNBD2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TTC27'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLCO1B3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MC1R'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('WNT2B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HS3ST4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RARS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CLDN10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATG3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PLEKHS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NCL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COL25A1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TPP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AR'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GTF2I'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DET1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VPS54'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AKR1D1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC17A7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IMPDH2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR10A5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ETV4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SHCBP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('URI1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IZUMO2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RHBDD3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PLAA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL58'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FREM1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CECR2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BUB1B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GSC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CALN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MSH5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNPC3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ETNK2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SSH1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PDCD11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HAUS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCDC170'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARMC9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AGAP3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SAFB2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE2D3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZSWIM2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LY6K'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AP4S1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UTS2R'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KICS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CRYZ'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYP1B1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCNK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TRIM3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SDHAF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SMPD2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE2V2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TSPYL5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BAIAP2L2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CBR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZFP41'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FOXN4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MTG2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TREML2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ABCG4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARPC2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KIDINS220'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OBP2A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ERCC8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CKS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SPIN4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MTREX'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EEFSEC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AEBP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PITPNM2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPL13A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBXN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MYBL2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CAMK2N1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HSD3B7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CHCHD3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATP6V1G1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCGF6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OAZ1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FCGR1A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HDAC2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LYAR'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BBS9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAPT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATG10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HUS1B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PUM1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('APAF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SPATA6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAST3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MOCS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UGP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BARX2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('XIRP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HMGN4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE2E1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C3orf52'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CD81'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ADCY8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COMMD3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('STX8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KCNMB3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCDC172'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SFXN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RHBDD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RUNX2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DFFA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LAYN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RIC1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FBXO6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MTMR9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE2S'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRELID2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCSK2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TYRO3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ADA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCDC66'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SEC24D'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RSPH14'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HHAT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GCAT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NLRP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF568'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GABRA6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LARS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SYT5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HOXC8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAB6A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PAGE5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF707'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SREBF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC25A21'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PTPN23'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TCERG1L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GPC3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HIBCH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MICAL3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NOTCH2NLA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DSCAM'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDC6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF44'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('USE1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KRT16'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VAMP5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GSX1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('WDR62'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAP4K5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GATAD2B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PAFAH1B1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NLGN3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GRK6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CEP57L1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UTP25'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C14orf93'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SUDS3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATG5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATXN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AMHR2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NFXL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TPD52L1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDK2AP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SS18'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DAXX'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KCNMA1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MSH3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NPFFR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TNK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CERT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ADAT2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SAMD4A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNF144B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HAUS4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATE1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF5A2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POLR2I'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SERPINB7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('STIL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SOX7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CARD18'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IGSF9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBQLN3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KRT6C'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FBXO32'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('H2AC17'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM126A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPIP5K1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NDST1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARFGAP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF2AK3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FBN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE2U'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF764'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GPANK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZBED3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TCEAL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF250'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IL2RB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DCLK2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MPZL2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KCNC4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('STT3A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GALNT14'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IL12A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ANKRD26'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TAF6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PSMC6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMED2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM20A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FDXR'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SIX5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCDHB11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAD51AP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PTCH1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SON'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PLPP5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FGF5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCDHA3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IL5RA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL18'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM95'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IGFBP3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TDRD10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CASP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OTUB2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MMAA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SEM1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRRX2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ITLN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('WDR83'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SPACA7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RHBDF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POLDIP3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PIGV'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('WDR91'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CARD16'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NEDD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ADAMTS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MPO'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AGO4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KRT79'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATP2B3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POLB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATXN3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PSMG3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AKT2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL47'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZCCHC13'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ILF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MMADHC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SULF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OSBPL9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ASB10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PTCHD4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF2AK2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CPPED1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DCAF12'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCRT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PLIN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LIN52'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE2NL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('USP37'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EMC6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FBXL16'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PEG3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPP1R1C'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MS4A10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DDX52'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC43A3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IMP3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CD2BP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PARN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDH26'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ADAT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATG2A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COG8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('B3GALNT2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GRM5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCDC169'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAP3K9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DHODH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EYA4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRSS33'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UGCG'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PI15'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATG7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ACCSL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MFSD2B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('WDR19'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR5H14'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCLO'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATG12'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF117'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EVI5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNF213'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPS15'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRKAG3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRKAR1A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KRT17'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ELOVL7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KCTD20'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('S100A12'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BNIP3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM216B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TUBE1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SKA1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PROK2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCNH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MOGAT2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ASB7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('INPP5A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ANKLE1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HOXA13'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KCTD3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BOK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CKAP2L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCDC27'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC7A13'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LRRC10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDR2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MARS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE2M'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF696'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDKN1A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNF13'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IFT57'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPP6R3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GLI2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPL7A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NIFK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAPK10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C14orf119'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KLF16'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SP3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CD6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('P2RX6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NEU1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DDX50'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HNRNPU'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPP1R15B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ADAMTS18'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NUP37'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DNAJB3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MIGA1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RIPK4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EME2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EFHC1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AMELX'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYP3A4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EPB41L4A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PSMD3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC9A3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SGTB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RHOC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYP2J2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TRIM21'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PROM1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF69'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LRRN4CL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMX4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FMO2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CUL4B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PARD6B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DAZL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C11orf91'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM167A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SRP54'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRSS27'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GTPBP8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CPNE5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYB5R1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPRD1B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GRK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OXGR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RWDD4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCFD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FLII'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PLIN3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TEAD3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM114A1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM207'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF260'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RBMX2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYLC1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RC3H1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM222B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF4E2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KBTBD3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CD1C'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCDHGA9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FLT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GADD45GIP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PIKFYVE'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PLEKHG2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PML'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LIG4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PAM16'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CMTR2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAB9A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ABRACL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FBXO48'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MTPAP'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HERC1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PANK3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZBTB8OS'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DCAF16'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DNM1L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ITPRIP'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NFATC2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('QSOX2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PHETA2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CRY2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DLG1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POLR2D'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MROH6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SERPINB11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RFX4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ANAPC7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CNTN4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM131'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DNAH5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TRPM6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HOXA11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TUBGCP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NELFE'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HARS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EBAG9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DNAJB1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF708'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ISCA1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GCN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('XKR6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IQCH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IFIT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CTSZ'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MYBPC3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TRIM52'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POTEH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BDP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('B4GALNT3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('WIPF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RUFY3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FGD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ENPP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MDC1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CENPU'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR2T8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LZTS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CIDEB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UTP6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RBPJL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ACOT11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ITPRID1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CTU1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IFT46'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZDHHC17'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAB33B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HEATR6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CC2D1B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF2B2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF2A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CHRNA5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DNAJC30'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPS6KA2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDX2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CANT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AOC3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TM9SF4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNF41'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCDHAC2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('STYK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CRISP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AMH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MIA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HSPA9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SH2D3A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PKLR'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EGFL8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PIK3CD'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('INSL6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MDFI'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CHMP4B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CRYGS'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMCC3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('H1-8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HUNK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MKI67'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ACTR3C'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POLR2H'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TNRC18'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OSBPL11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SAMD7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ODF2L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM17'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('H3-3B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DLX2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SRPK3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TNFRSF10B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF638'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDKL5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('XRRA1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MTFMT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CASP6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PAAF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VGLL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RBL2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TRIM46'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRSS37'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IL1B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MEP1A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C4orf33'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RBAK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HSPA5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AIF1L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATG4C'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRKCZ'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HRK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DACH1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCN4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('USP17L12'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PHOX2A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FANCL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('REV1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM61'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DNAJC5G'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FERD3L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE2D1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KRT15'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCDHB10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UTP23'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GAPVD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ERCC6L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('USP47'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM72A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HDGFL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POU6F2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('INIP'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DENND4B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL14'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RBM33'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MDH1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LAMP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TOMM6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RB1CC1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CEP63'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MMD2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OXR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BATF3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATG9B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM167A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EDAR'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IARS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CTDNEP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RFPL2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CHDH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NAGK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C6orf136'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GIPC2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('APOBEC3A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NEUROD6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IL36RN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCDHB13'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAP2K5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NT5C3B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HSPD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FMR1NB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SF3B6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NMNAT3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRKD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CSNK1E'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SYCP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPIC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HSF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBXN7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAB29'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARL5B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CAB39'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HTN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TXNL4A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCFD2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZBTB2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCDHGB2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TLCD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RTEL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GPR33'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MFSD8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SULT1E1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EVPL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MEDAG'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DGKH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UNKL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLITRK2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR4F6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NFE2L2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NUDCD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GRK4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NR5A1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DOLPP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FMN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYLD'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POLDIP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM174C'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNASEH2B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TOMM40'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('H2BC8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF716'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SDE2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GPSM2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ANKRD20A1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLAIN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF254'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CARD9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CHRNB3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C8orf76'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POMC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C2orf78'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UNC5D'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE2K'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZAN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PALMD'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DENND4C'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IPO11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VNN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PYROXD2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NRDC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HNRNPUL2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PAQR6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR2B11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MFHAS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NFIX'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AMPH'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SPINK2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ADRA1D'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OCEL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BET1L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCN1A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRGPRF'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KCND1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ANKMY1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RCC1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UHRF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPS22'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF4E'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRDM16'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ALYREF'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LACC1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PLEKHD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MCM6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM69'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PBOV1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SPTSSA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TXNDC16'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DDR2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C12orf43'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BICC1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KLHL25'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM132D'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ACBD6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COG6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GRSF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RGS22'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF521'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('APRG1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IMPG2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CSNK1G1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CLIP4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RABL3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KRT76'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SRPX2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAB41'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('WT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF37A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RABGGTB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FBXW10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CGREF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TEC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BAK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FURIN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NUP214'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POLK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCL24'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DDX46'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR1N2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LRRC31'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IL22RA2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAD51D'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KPNA2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MTR'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MSTN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MICU1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR51B5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SHCBP1L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LCE1B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GCM1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM50A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GCNT4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF485'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SF3B3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NCAPG'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZMYND15'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CACUL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR1S2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EMC4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C9orf43'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SEPTIN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MTFR2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MFSD3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TCF20'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('USP21'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GUCA1A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DDX27'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('YJU2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HNRNPCL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TJAP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPS6KA4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CTNNA1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF827'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TRAPPC9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RHPN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATP11C'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PINX1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCDC90B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC2A11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('L3MBTL2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM134'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MOB1B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDC27'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM123'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CAPG'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MATN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PBK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RIPOR3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CLCN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RELT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BCL2A1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM259'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RADIL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC39A11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF133'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IL1RL2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KY'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF341'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FIGN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('STX3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPS15A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EPS8L2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('STYXL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RMI2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CHMP2A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TLCD5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR10G4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PDZD9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SYPL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNF166'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('H4C11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ACSM4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VGLL2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SRD5A1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CRISP3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAB5IF'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCDC83'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TRIM7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IKZF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZFP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BMP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ST7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CASS4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MSRB2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARF4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCPEP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SPATA6L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BBS5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DUOXA2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AXDND1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HRH2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAP3K15'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SDR9C7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PSMD13'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DPAGT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ABTB3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DCUN1D2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SPATA4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF713'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR2T5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TBC1D8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SPATA46'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ABHD5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PARPBP'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KCNG4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SNX12'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF419'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARGFX'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BEND2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LAPTM5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR2A4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NOA1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PDS5B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNF39'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLITRK3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('XPO4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCLT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DNAH11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPFIA4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZBTB39'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DDX20'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OAF'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF789'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF705EP'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLITRK5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR5C1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GPR32'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LCN6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('USP42'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CXCL10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NWD2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF100'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CAVIN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FANCA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ASCC3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CLSTN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VPS36'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RASL10A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCARF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAPGEF2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ULBP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ALS2CL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZC4H2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GIMAP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RASL11A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MUC16'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GRIA1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SECISBP2L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VSIG4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MDM2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('USP8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLX4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AHCTF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FN3K'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('STX12'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HOGA1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE2I'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARHGAP23'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ACTR5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCDC14'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARID1B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF385D'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KHDC3L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RSBN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BCAR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BMAL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UBE3A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DZIP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DDX60L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SERPINA5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AP2B1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EFTUD2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('C19orf18'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ARID1A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HTR7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAP3K11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CBX2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GML'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PLEKHA6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPS9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SELENOT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HOXD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPL19'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NVL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('STAT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAT2A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RLF'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TUBG2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDC42EP5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PTPN14'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RAD9B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KIR2DL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GPAT3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BRD2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CISD3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TCAF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MAPKAPK5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CALR3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ETV3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SPRR1B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ABCD4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF695'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PBX1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NSUN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SRR'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TOPBP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GP5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GCNA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ERGIC2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SF3B1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MSLN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ILDR2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NGDN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NAIP'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CSTF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR2L13'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BARD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CHFR'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LYRM2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYTH4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMEM200A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDC16'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF608'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PKN1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MPZ'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCSK6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GRIP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CEBPE'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CES2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DAZ3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PDPK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPS7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RTL6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GCK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EPN3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF5B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NCOR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SPATA9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('STARD6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PROSER2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('THOC3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LGR5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNRF3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VXN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DNAJC17'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PAPLN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EXOSC10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF320'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF551'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('IRF2BP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FOXD3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TTN'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCDHB3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TOP2B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('OR51F2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MED19'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FAM241B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('S100Z'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('POLR2J2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SEC31B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DAD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RCHY1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PDE7A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DHX38'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GRIK1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('HUWE1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FBXO9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZFHX3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZFP69'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NCOA1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCAMP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPP4R3B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CCDC188'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NPAS2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DNASE1L3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MROH5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SERPINB3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('KCNV1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ANAPC10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TFPI2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MTA2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NRM'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AGTR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ACP4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NT5M'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('QARS1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TAPT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TOGARAM1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EIF3B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF570'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CDC20B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DNM3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FGFBP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SNX21'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PIGQ'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PSMD11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CHST8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CMTR1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF655'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TTC23'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CREG1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BTN2A2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SWT1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DDX42'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('DENND5B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('WRAP53'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ACOXL'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TSTD1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FMO5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF420'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PTPRZ1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PCDH18'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VPS39'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LDB1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF792'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('BIN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AHSA1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TXNL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF138'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FABP9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TMPRSS5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SH2D5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('B3GNT4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GSDMC'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('R3HDM2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NDUFB2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ANXA5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RNF8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF606'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NEDD8'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TTC39C'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NAA50'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PLA2G1B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('GPR157'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CAVIN3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PSMG4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ACYP1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRAMEF4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SCARF1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SLC9A7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COQ2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('COPS6'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('APOC3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MRPS33'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('LRRC34'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TAF10'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF74'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('AVL9'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ATP10A'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PATZ1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SAMD4B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PDLIM2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PTCD2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('UTS2B'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('ZNF511'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FANCM'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MYL11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('NDST4'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SYT14'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('RIN2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('MXRA7'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('FBXW11'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('TBC1D1'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('EMC2'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PELI3'), np.str_('RAB1B')], ['TP53', np.str_('PDGFRA'), np.str_('PITPNM3'), np.str_('RAB1B')], ['TP53', np.str_('EHF'), np.str_('PDGFRA'), np.str_('RAB1B')], ['TP53', np.str_('EHF'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('EHF'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('EHF'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('EHF'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('EHF'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('EHF'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('STARD6'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('ADAMTSL5'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CCDC82'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('PSKH2'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CYP3A4'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('MROH5'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('KBTBD3'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('DNAJC5G'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('HDAC9'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('APRG1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('GDPD4'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('DENND5B'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('OR52B2'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('UBE2I'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('TOMM6'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('PATE2'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('LRRN1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('KCTD3'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('KIF18A'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('R3HDM2'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('RGS22'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CSTF1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CHST8'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('PLEKHS1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('ANXA5'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('KLHL38'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('FAM167A'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('SHCBP1L'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('PDCD11'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('RSRP1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('NCOR1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('SYT14'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('AHCTF1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('KCNMA1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('TMEM61'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('NR1I2'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('EFL1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('XAGE3'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('RTL6'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CCDC171'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('UBE3B'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('OAF'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('FMR1NB'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('SH2D3A'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('FBXO9'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CYP2C18'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('MAPKAPK5'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('FBXW10'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('TMEM248'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('ANKRD20A1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('IKZF2'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('TBC1D1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('FOXR1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('OFCC1'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('CALML4'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('BMPR1A'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('BCL2A1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('PDE3A'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('CBR1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('ATG2A'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('UBC'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('ATG7'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('SH3GLB1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('RAB12'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('FLT3'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('RB1CC1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('STK4'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('MAP1LC3C'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('BNIP3'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('LAMP1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('SREBF2'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('PDGFRA'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('ESR1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('SKP2'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('GTSE1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('EIF4E'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('STX8'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('RNF213'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('HSF1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('CDKN1A'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('IMPDH1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('CRY2'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('PIK3R4'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('BCL2'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('STK24'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ENPP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ARIH1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('WNK4'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PARP2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('NFKB2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CHEK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('HIPK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('GTSE1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('HACE1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CSNK1E'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('STK4'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('EIF4E'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PSMA4'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SECISBP2L'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('HCK'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MMP25'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MSI2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SMPD2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ABCB7'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('AOC3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('HK2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('EPHB1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('GET3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MDM2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('FGFR3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CCT2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CDC26'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('COPS5'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('KARS1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PAM16'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('UBE2C'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CDC73'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CAMK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('BUB1B'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CDKN1A'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CEP152'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TKT'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PFKM'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MAPK12'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PTGS1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('EIF3H'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RECQL4'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CYP1B1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PPARA'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SCPEP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MAPK10'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ACTL6A'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CTNNBL1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('AURKAIP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('VPS54'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('AK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('H2AC8'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('IL2RG'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TSPO'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ABCC2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CRYBB1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('GINS1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CDC25B'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PAFAH1B1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PRKAG3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TNK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ANAPC13'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('NAGK'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CSK'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MDM4'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MASP2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PPP1CC'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('HDAC2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PER1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('BUB3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ASCC3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('EIF3M'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('COX7C'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('POLB'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RAD9A'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RPS18'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('XRCC6'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('FLT1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RPS6KA2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('UGCG'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SLC22A1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('COG3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('YJU2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('FBN1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('POLR2A'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PDE3A'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('GRK6'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('BOD1L1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('AMHR2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('BMAL1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ESPL1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RPL13A'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RIC1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MAP2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('EIPR1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('LARS1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CRYBB2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SLC5A5'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SYNE2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('STYXL1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SS18L1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CRY2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('HMGCR'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RNF213'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CMAS'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CKS2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('UGP2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RET'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TIPARP'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('AANAT'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('POLD4'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CASP9'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PTPRE'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PTPA'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('NR1I2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ATR'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('E2F1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ERCC6L'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RIPK4'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('NLRP2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RPS10'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PIK3CD'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CDK2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('BRIP1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SNRK'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('NPR2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('APC'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('FBP2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('UBE3A'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SEC23IP'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ARF4'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PLA2G2E'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ZNF517'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CENPK'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MCPH1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('IGFBP3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('BRD2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('INTS3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RXYLT1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ERBB3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PPM1D'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PSMD9'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ADAMTS2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('POMC'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MCM5'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SDHB'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('EIF2B2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PRKCZ'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ASPSCR1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RPS5'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PAPSS1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ATM'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PTCH1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PRKD1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('FURIN'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('PML'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('TK2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CFLAR'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('HSF1'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ABCC4'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('ERGIC2'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('XPA'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('CDKL5'), np.str_('RAB1B')], ['TP53', np.str_('MAP4K5'), np.str_('RBM12'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('SEPTIN1'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('PDGFRB'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('SIK1'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('XRCC5'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('PPME1'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('CTNNB1'), np.str_('EP300'), np.str_('RAB1B')], ['TP53', np.str_('CTNNB1'), np.str_('EZH2'), np.str_('RAB1B')], ['TP53', np.str_('CTNNB1'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('CTNNB1'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('CTNNB1'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('CTNNB1'), np.str_('MAPK3'), np.str_('RAB1B')], ['TP53', np.str_('CTNNB1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('CTNNB1'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('CTNNB1'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('KDM5C'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('KDM5C'), np.str_('PBRM1'), np.str_('RAB1B')], ['TP53', np.str_('KDM5C'), np.str_('PDGFRA'), np.str_('RAB1B')], ['TP53', np.str_('KDM5C'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('KDM5C'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('KDM5C'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('KDM5C'), np.str_('MSH6'), np.str_('RAB1B')], ['TP53', np.str_('KDM5C'), np.str_('NF1'), np.str_('RAB1B')], ['TP53', np.str_('KDM5C'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('KDM5C'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('KDM5C'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('EYA3'), np.str_('EYA4'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('ABCA3'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('TMEM165'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('EGFR'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('BLM'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('FBXW7'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('ARMH3'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K15'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('RAB12'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('RAB33B'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('RPS6KB1'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('RB1'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('STK4'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('ATG7'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('PIK3R4'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('COG7'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('STX8'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('BRCA1'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('ATG12'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('SREBF2'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('SKP2'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('SH3GLB1'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('RB1CC1'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('PLK1'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('PIK3CD'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('SMO'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('STX3'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('TRIM21'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('VPS52'), np.str_('RAB1B')], ['TP53', np.str_('PIK3CA'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('NMNAT1'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('VEGFA'), np.str_('VHL'), np.str_('RAB1B')], ['TP53', np.str_('VEGFA'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('SCYL3'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('GSTP1'), np.str_('RADIL'), np.str_('RAB1B')], ['TP53', np.str_('GSTP1'), np.str_('ZNF716'), np.str_('RAB1B')], ['TP53', np.str_('GSTP1'), np.str_('RHOBTB3'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('POLR2L'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('CYP3A7'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('CYP2C9'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('TOP1'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('SIRT1'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('TRIM52'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('AK2'), np.str_('BRCA2'), np.str_('RAB1B')], ['TP53', np.str_('CDC37L1'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('VRK2'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('VRK2'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('VRK2'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('GABARAP'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('MAP1LC3B'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('MAP1LC3A'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('PI4K2A'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('PINK1'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('RB1CC1'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('IRGM'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('PIK3R4'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('PIK3CB'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('PIK3C3'), np.str_('RAB1B')], ['TP53', np.str_('FOXO1'), np.str_('GABARAPL1'), np.str_('RAB1B')], ['TP53', np.str_('ENPP4'), np.str_('HRAS'), np.str_('RAB1B')], ['TP53', np.str_('ENPP4'), np.str_('ENPP6'), np.str_('RAB1B')], ['TP53', np.str_('CKM'), np.str_('RELA'), np.str_('RAB1B')], ['TP53', np.str_('CKM'), np.str_('PTPRJ'), np.str_('RAB1B')], ['TP53', np.str_('CKM'), np.str_('PIK3CG'), np.str_('RAB1B')], ['TP53', np.str_('CKM'), np.str_('XIAP'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SLC35A1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SMC5'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MZT1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NFE2L1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('UQCRB'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MTBP'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TIMM23B'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NCBP1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PWP1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PSPH'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TIMM22'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RPL35'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('GFI1B'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RPS28'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TRMT5'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('UBA2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SLC7A6OS'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('GPN3'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PRPF18'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('INTS3'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('POLN'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RPLP1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TUBGCP3'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TTI2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SNRPB'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RPLP2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TUBGCP5'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TUBB'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('METAP2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SNRNP70'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TBC1D31'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NSMCE1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PSMB7'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RPL3'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PNN'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PDRG1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SRSF7'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SNRPG'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('ZWINT'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SRFBP1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('FDPS'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RFC4'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('GEMIN4'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('IMPDH2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MRPL13'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MTPAP'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MRPS18A'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TCF25'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RTF1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SCO1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NACA'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('INTS5'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MRPL50'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('VPS29'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NUDC'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RNGTT'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NUP43'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RPP30'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MRPL22'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('ZMAT5'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SKIC8'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('FH'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RPL8'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PAFAH1B1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SDHC'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TBCB'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('ZMAT2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PGD'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NIP7'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MTHFD2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NDUFB2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NAA50'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SFXN1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('GNPAT'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PDSS2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MRPS30'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NCAPD2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('ROMO1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NSMCE4A'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MARS2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('FAM50A'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MAPKAP1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MRPS11'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MRPS9'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TIMM9'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('WARS2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SEC22B'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('LUC7L3'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SNIP1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('HSCB'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RBM17'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MED23'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('WDR7'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TUBGCP4'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RINT1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RPL11'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NOL9'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('XPO1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MED9'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MRPL10'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('HCCS'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MBTPS2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MED14'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SMNDC1'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MRPL15'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PMPCB'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SKIC2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('NDUFA8'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('WDR36'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PRMT5'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('SEH1L'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MRPS16'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('RPL30'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('REXO2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MAX'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('PRIM2'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('TRA2B'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MRPL33'), np.str_('RAB1B')], ['TP53', np.str_('EXOSC9'), np.str_('MAT2A'), np.str_('RAB1B')], ['TP53', np.str_('SULT1A2'), np.str_('SULT1E1'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('PDHX'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('CASP3'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('CCNH'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('PIK3CA'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('MTOR'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('PPWD1'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('AURKA'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('TRIM52'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('ZNF830'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('PFKL'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('EXD2'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('NUDT1'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('SLC39A9'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('RPA3'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('MTRES1'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('TAF2'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('CIT'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('TUBG1'), np.str_('RAB1B')], ['TP53', np.str_('TTK'), np.str_('MEN1'), np.str_('RAB1B')], ['TP53', np.str_('STYK1'), np.str_('RAB1A'), np.str_('RAB1B')], ['TP53', np.str_('STYK1'), np.str_('RAB1B')], ['TP53', np.str_('STYK1'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('STYK1'), np.str_('ABCG2'), np.str_('RAB1B')], ['TP53', np.str_('STYK1'), np.str_('CYP2D6'), np.str_('RAB1B')], ['TP53', np.str_('STYK1'), np.str_('DMXL1'), np.str_('RAB1B')], ['TP53', np.str_('STYK1'), np.str_('CYP2C19'), np.str_('RAB1B')], ['TP53', np.str_('STYK1'), np.str_('NTRK1'), np.str_('RAB1B')], ['TP53', np.str_('STYK1'), np.str_('ORM1'), np.str_('RAB1B')], ['TP53', np.str_('MAP3K7'), np.str_('BCL2'), np.str_('RAB1B')], ['TP53', np.str_('RAF1'), np.str_('WDR48'), np.str_('RAB1B')], ['TP53', np.str_('CDC7'), np.str_('MYBL2'), np.str_('RAB1B')], ['TP53', np.str_('CDC7'), np.str_('STIL'), np.str_('RAB1B')], ['TP53', np.str_('CDC7'), np.str_('TOPBP1'), np.str_('RAB1B')]]

SyntaxError: invalid character '→' (U+2192) (967082447.py, line 1)

In [ ]:
The Kernel crashed while executing code in the current cell or a previous cell. 
Please review the code in the cell(s) to identify a possible cause of the failure. 
Click <a href='https://aka.ms/vscodeJupyterKernelCrash'>here</a> for more info. 
View Jupyter <a href='command:jupyter.viewOutput'>log</a> for further details.

SyntaxError: invalid syntax (2791278521.py, line 1)

In [ ]:
flowers_images_path: '/content/Text-to-Image-Synthesis/flowers_dataset/jpg/'


flowers_embedding_path: '/content/flowers_dataset/text_c10/'
flowers_text_path: '/content/flowers_dataset/text_c10/'

flowers_dataset_path: '/content/flowers_dataset/flowers.hdf5'

flowers_val_split_path: '/content/flowers_dataset/valclasses.txt'
flowers_train_split_path: '/content/flowers_dataset/trainclasses.txt'
flowers_test_split_path: '/content/flowers_dataset/testclasses.txt'